In [ ]:
_ = !pip install --upgrade gupload
_ = !pip install ta 
_ = !pip install pytictoc
_ = !pip install -q -U keras-tuner

# Import library

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# IMPORTS
import pandas as pd
import math
import os.path
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# from bitmex import bitmex
#from binance.client import Client
from datetime import timedelta, datetime
from dateutil import parser
from tqdm import tqdm_notebook #(Optional, used for progress-bars)

# save to google drive
from pydrive.auth import GoogleAuth
from google.colab import auth
from google.colab import drive

import pandas as pd
import numpy as np
import pickle
from numpy.polynomial.polynomial import polyfit
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
from datetime import datetime, timedelta
import random
import warnings
from pytictoc import TicToc
t = TicToc() #create instance of class

# TA package for technical indicator
import ta as ta
from ta import add_all_ta_features
from ta.utils import dropna

# Scipy
import scipy
from scipy.signal import argrelmin, argrelmax
from scipy.signal import savgol_filter

# Keras / Tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, BatchNormalization, Input, Conv1D, MaxPooling1D, Flatten, GRU, Dropout, Bidirectional, Concatenate
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.regularizers import l2
from time import time
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from keras import backend as K

#Sklearn
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler, MinMaxScaler, QuantileTransformer, OneHotEncoder
#from sklearn.metrics import classification_report, precision_recall_curve, plot_precision_recall_curve


# import yfinance as yf

# fix seed for all random function

# seed_value = 42 # seed for all random function
# os.environ['PYTHONHASHSEED']=str(seed_value)
# random.seed(seed_value)
# np.random.seed(seed_value)
# tf.random.set_seed(seed_value)


# Configure a new global `tensorflow` session
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.config.experimental.enable_op_determinism()
tf.compat.v1.keras.backend.set_session(sess)

seed_value = 42 # seed for all random function
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
tf.keras.utils.set_random_seed(0) 


Mounted at /content/gdrive


# Trade

In [ ]:
def trade_by_model(temp, start_m=100, hold=0):
  m = start_m
  count_buy = 0
  count_sell = 0

  for index, row in temp.iterrows():
    status_buy = 0
    status_sell = 0

    # if row['sum_p'] >= 3: 
    #   status_buy = 1
    # elif row['sum_n'] < 3:
    #   status_sell = 1
    if row['predict_1'] > 0:
      status_buy = 1
    elif row['predict_1'] < 0:
      status_sell = 1

    # if (row['smoothed_return_close_predict_n1'] > 0) & (row['smoothed_return_close_predict_n2'] > 0) & (row['smoothed_return_close_predict_n3'] > 0):
    #   status_buy = 1
    # elif (row['smoothed_return_close_predict_n1'] < 0) & (row['smoothed_return_close_predict_n2'] < 0) & (row['smoothed_return_close_predict_n3'] < 0):
    #   status_sell = 1

    if status_buy == 1:
      if hold == 1:
        pass
      else:
        count_buy = count_buy + 1
        temp.loc[index:]['buy'] = 1
        temp['buy'] = np.where(temp['Time']==row['Time'],1,temp['buy'])
        current_price = row['Close']
        hold = 1
        

    if status_sell == 1:
      if hold == 0:
        pass
      else:
        count_sell = count_sell +1
        temp.loc[index:]['sell'] = 1
        temp['sell'] = np.where(temp['Time']==row['Time'],1,temp['sell'])
        m = (row['Close']/current_price)*m
        hold = 0

    temp['port'] = np.where(temp['Time']==row['Time'],m,temp['port'])
    temp['hold'] = np.where(temp['Time']==row['Time'],hold,temp['hold'])
  # print(count_buy)
  # print(count_sell)
  return ((m-start_m)/start_m)*100, temp, m

In [ ]:
class Portfolio:
    def __init__(self, initial_capital = 1000):
        self.portfolio = {}
        self.cash = initial_capital
        self.max_port_value = initial_capital
        self.max_port_value_time = pd.Timedelta(0)
    
    # position
    def add_position(self, position,  # position = 'long' or 'short'
                     open_time, open_price, open_value):
        # check cash
        if self.cash < open_value:
          print('ERROR - can afford lesser amount')
          open_value = self.cash
          if open_value == 0:
            print('ERROR - cannot afford')
            return

        # check opposite position
        if position == 'long' and self.is_position('short'):
          print('ERROR - caanot open long position while there is exposure of short position')
        elif position == 'short' and self.is_position('long'):
          print('ERROR - caanot open short position while there is exposure of long position')

        self.cash -= open_value
        self.portfolio[open_time] = Trade(position, open_price, open_value)

    
    def close_position(self, position, close_size, close_price, close_time, trd_fee_frac, hld_fee_frac_hr):
        
        close_trade_list = []
        while close_size > 0:
          # Find open time to close (FIFO)
          if len(self.portfolio.items()) > 0:
            open_time = min([key for key, value in self.portfolio.items()])
          else:
            print('ERROR - No opening position to close')

          # Close whole trade
          if close_size >= self.portfolio[open_time].size:
            # find size to close
            trade_close_size = self.portfolio[open_time].size
          # Close part of trade
          else:
            # find size to close
            trade_close_size = close_size
            
          # to correct negative price
          close_price = max(0, close_price)
          
          close_trade_summary, open_value, PnL = self.portfolio[open_time].close(trade_close_size, close_price, close_time, open_time, trd_fee_frac, hld_fee_frac_hr)

          # update port
          close_size -= trade_close_size
          if position == 'long':
            self.cash += open_value + PnL # trade_close_size * close_price
          elif position == 'short':
            self.cash += open_value + PnL # trade_close_size * (self.portfolio[open_time].open_price * 2 - close_price)

          close_trade_summary.append(self.cash)
          close_trade_summary.append(self.get_total_opening_value())
          

          # delete trade
          if self.portfolio[open_time].size <= 0: 
            del self.portfolio[open_time]
          
          # append trand summary
          close_trade_list.append(close_trade_summary)
          
        return close_trade_list
      
    def get_total_opening_value(self):
        return sum([value.open_price * value.size for key, value in self.portfolio.items()])
      

    def is_position(self, position):
        return self.get_size(position) > 0

    def update(self, close, high, low, time, trd_fee_frac, hld_fee_frac_hr):

        # update stat of all trade
        self.update_trade_stat(close, high, low, time, trd_fee_frac, hld_fee_frac_hr)

        # get port value
        port_value = self.get_total_unrealized_trade_value() + self.cash

        # to correct negative price
        port_value = max(0, port_value)

        if port_value >= self.max_port_value:
          self.max_port_value = port_value
          self.max_port_value_time = time
        
        dd = min((port_value - self.max_port_value) / self.max_port_value, 0)
        dd_dur = time - self.max_port_value_time

        port_size = self.get_total_size()
        return [time, port_value, port_size, dd, dd_dur]

    def get_total_unrealized_trade_value(self):
        return sum([trade.value for key, trade in self.portfolio.items()])

    def update_trade_stat(self, close, high, low, time, trd_fee_frac, hld_fee_frac_hr):
        for k, trade in self.portfolio.items():
            trade.update_stat(k, close, high, low, time, trd_fee_frac, hld_fee_frac_hr)

    def get_total_size(self):
        long_size = self.get_size('long')
        short_size = self.get_size('short')
        return long_size - short_size

    def get_size(self, position):
        return sum([value.size for key, value in self.portfolio.items() if value.position == position])

class Trade:
    def __init__(self, position, open_price, open_value):
        
        self.position = position
        self.open_price = open_price
        self.price = open_price
        self.value = open_value
        self.size = open_value / open_price
        self.maxDD = 0


    def close(self, trade_close_size, close_price, close_time, open_time, trd_fee_frac, hld_fee_frac_hr):
        
        trade_close_size = trade_close_size
        open_price = self.open_price
        close_value = close_price * trade_close_size
        open_value = open_price * trade_close_size
        trd_fee = trd_fee_frac * (open_value + close_value)
        trd_fee = 0
        hld_dur = close_time - open_time
        hld_dur_hr = (hld_dur).total_seconds() / 3600
        hld_fee = hld_fee_frac_hr * hld_dur_hr * (open_value + close_value)/2

        if self.position == 'long':
          PnL = close_value - open_value - trd_fee - hld_fee
        elif self.position == 'short':
          PnL = open_value - close_value - trd_fee - hld_fee
        
        maxDD = self.maxDD
        return_frac = PnL / open_value

        win = True if return_frac > 0 else False


        # update trade
        self.size = self.size - trade_close_size 
        return [self.position, open_time, close_time, open_price, close_price, trade_close_size, hld_dur, trd_fee, hld_fee, PnL, maxDD, return_frac, win], \
                open_value, PnL

    def update_stat(self, open_time, close, high, low, time, trd_fee_frac, hld_fee_frac_hr):
        self.price = close
        current_value = close * self.size
        open_value = self.open_price * self.size
        trd_fee = trd_fee_frac * open_value # trd_fee_frac * (open_value + current_value)
        hld_dur_hr = (time - open_time).total_seconds() / 3600
        hld_fee = hld_fee_frac_hr * hld_dur_hr * (open_value + current_value)/2
        
        if self.position == 'long':
          PnL = current_value - open_value - trd_fee - hld_fee

        elif self.position == 'short':
          PnL = open_value - current_value - trd_fee - hld_fee
        
        self.value = open_value + PnL # trade value
        self.maxDD = min(PnL / open_value, self.maxDD)

def Backtest(df, start, end, col_name = None, initial_capital = 100, 
             trd_fee_frac = 0.000, hld_fee_frac_hr = 0# 2.5e-05 # hld_fee_frac_hr = 2.5e-05 = 0.02 %/8hr
             ):
  if col_name == None: col_name = ['Open', 'High', 'Low', 'Close', 'Long_Position', 'Short_Position', 'Long_Price', 'Short_Price']

  df = df.rename(columns={col_name[0]: 'Open',
                          col_name[1]: 'High',
                          col_name[2]: 'Low',
                          col_name[3]: 'Close',
                          col_name[4]: 'Long_Position',
                          col_name[5]: 'Short_Position',
                          col_name[6]: 'Long_Price',
                          col_name[7]: 'Short_Price'})

  # defind portfolio
  PORTFOLIO = Portfolio(initial_capital = initial_capital)
  PORTFOLIO_curve = []
  PORTFOLIO_trades = []
  PORTFOLIO_stats = {}

  for i in range(len(df)):
    
    # update portfolio curve
    curve = PORTFOLIO.update(close = df['Close'][i], high = df['High'][i], low = df['Low'][i], 
                             time = df.index[i], trd_fee_frac = trd_fee_frac, hld_fee_frac_hr = hld_fee_frac_hr)
    PORTFOLIO_curve.append(curve) 

    # check close position
    if (df['Long_Position'][i] < df['Long_Position'][i-1]):
      # close long position

      # calculate fraction for close position from current size
      close_size = ((df['Long_Position'][i-1] - df['Long_Position'][i]) / df['Long_Position'][i-1]) * PORTFOLIO.get_size('long')

      # close long position
      close_trade_list = PORTFOLIO.close_position('long', close_size = close_size, close_price = df['Long_Price'][i], close_time = df.index[i], 
                                          trd_fee_frac = trd_fee_frac, hld_fee_frac_hr = hld_fee_frac_hr)
      

      for j in close_trade_list:
        PORTFOLIO_trades.append(j)

    if (df['Short_Position'][i] < df['Short_Position'][i-1]):
      # close short position

      # calculate fraction for close position from current size
      close_size = ((df['Short_Position'][i-1] - df['Short_Position'][i]) / df['Short_Position'][i-1]) * PORTFOLIO.get_size('short')

      # close long position
      close_trade_list = PORTFOLIO.close_position('short', close_size = close_size, close_price = df['Short_Price'][i], close_time = df.index[i], 
                                          trd_fee_frac = trd_fee_frac, hld_fee_frac_hr = hld_fee_frac_hr)
      
      # record all closed trades
      for j in close_trade_list:
        PORTFOLIO_trades.append(j)

    # check open postion
    if (i == 0 and df['Long_Position'][i] > 0) or \
        (df['Long_Position'][i] > df['Long_Position'][i-1]):
      # open long position

      # calculate fraction for open position from remaining cash
      prev_long_position = df['Long_Position'][i-1] if i != 0 else 0
      open_value = (df['Long_Position'][i] - prev_long_position) * (PORTFOLIO.cash + PORTFOLIO.get_total_opening_value())

      # open long position
      PORTFOLIO.add_position('long', open_time = df.index[i], open_price = df['Long_Price'][i], open_value = open_value)

    if (i == 0 and df['Short_Position'][i] > 0) or \
        (df['Short_Position'][i] > df['Short_Position'][i-1]):
      # open short position

      # calculate fraction for open position from remaining cash
      prev_short_position = df['Short_Position'][i-1] if i != 0 else 0
      open_value = (df['Short_Position'][i] - prev_short_position) * (PORTFOLIO.cash + PORTFOLIO.get_total_opening_value())
      
      # open short position
      PORTFOLIO.add_position('short', open_time = df.index[i], open_price = df['Short_Price'][i], open_value = open_value)

  # portfolio stat summary
  PORTFOLIO_stats['_curve'] = pd.DataFrame(data = PORTFOLIO_curve, columns=['timestamp', 'port_value', 'port_size', 'dd', 'dd_dur'])
  PORTFOLIO_stats['_curve'] = PORTFOLIO_stats['_curve'].set_index('timestamp')
  
  PORTFOLIO_stats['_trades'] = pd.DataFrame(data = PORTFOLIO_trades, columns=['position', 'open_time', 'close_time', 'open_price', 
                                                                              'close_price', 'trade_close_size', 'hld_dur', 'trd_fee', 'hld_fee', 
                                                                              'PnL', 'maxDD', 'return_frac', 'win',
                                                                              'remain_cash', 'total_opening_value'])
  PORTFOLIO_stats['start'] = df.index[0]
  PORTFOLIO_stats['end'] = df.index[-1]
  PORTFOLIO_stats['dur'] = df.index[-1] - df.index[0]
  PORTFOLIO_stats['init_capital'] = initial_capital
  PORTFOLIO_stats['final_capital'] = PORTFOLIO_stats['_curve']['port_value'][-1]
  PORTFOLIO_stats['return_frac'] = PORTFOLIO_stats['final_capital'] / PORTFOLIO_stats['init_capital'] - 1
  PORTFOLIO_stats['return_frac_ann'] = ((1 + PORTFOLIO_stats['return_frac']) ** (365/PORTFOLIO_stats['dur'].days)) - 1

  # Asset STD
  # std_per_bar = df['Close'].pct_change().std()
  # Port STD
  std_per_bar = PORTFOLIO_stats['_curve']['port_value'].pct_change().std()
  no_bar_in_yr = 1 / ((df.index[-1] - df.index[0]).days/365/df.shape[0])

  PORTFOLIO_stats['volatility_ann'] = std_per_bar * (no_bar_in_yr ** 0.5)
  if PORTFOLIO_stats['volatility_ann'] != 0:
    PORTFOLIO_stats['sharpe_ratio_ann'] = (PORTFOLIO_stats['return_frac_ann'] - 0) / PORTFOLIO_stats['volatility_ann']
  else:
    PORTFOLIO_stats['sharpe_ratio_ann'] = np.nan
  PORTFOLIO_stats['port_mdd_frac'] = min(PORTFOLIO_stats['_curve']['dd'])
  PORTFOLIO_stats['port_mdd_dur'] = min(PORTFOLIO_stats['_curve']['dd_dur'])

  # No Trade
  if PORTFOLIO_stats['_trades'].shape[0] == 0:
  
    PORTFOLIO_stats['trade_mdd_frac'] = 0
    PORTFOLIO_stats['trade_P10mdd_frac'] = 0
    PORTFOLIO_stats['win_rate'] = 0
    PORTFOLIO_stats['no_trades'] = 0
    PORTFOLIO_stats['avg_hld_dur'] = pd.Timedelta(0)
    PORTFOLIO_stats['max_hld_dur'] = pd.Timedelta(0)
    PORTFOLIO_stats['reward_to_risk'] = 0
    PORTFOLIO_stats['expectency_frac'] = 0

  else:

    PORTFOLIO_stats['trade_mdd_frac'] = min(PORTFOLIO_stats['_trades']['maxDD'])
    PORTFOLIO_stats['trade_P10mdd_frac'] = np.percentile(PORTFOLIO_stats['_trades']['maxDD'], 10)
    PORTFOLIO_stats['win_rate'] = PORTFOLIO_stats['_trades']['win'].sum()/PORTFOLIO_stats['_trades']['win'].count()
    PORTFOLIO_stats['no_trades'] = PORTFOLIO_stats['_trades'].shape[0]
    PORTFOLIO_stats['avg_hld_dur'] = PORTFOLIO_stats['_trades']['hld_dur'].mean()
    PORTFOLIO_stats['max_hld_dur'] = PORTFOLIO_stats['_trades']['hld_dur'].max()
    
    reward = PORTFOLIO_stats['_trades']['PnL'][PORTFOLIO_stats['_trades']['PnL'] > 0].sum()
    risk = abs(PORTFOLIO_stats['_trades']['PnL'][PORTFOLIO_stats['_trades']['PnL'] < 0].sum())

    if risk != 0:
      PORTFOLIO_stats['reward_to_risk'] = reward / risk
    elif PORTFOLIO_stats['_trades']['PnL'][PORTFOLIO_stats['_trades']['PnL'] > 0].sum() > 0:
      PORTFOLIO_stats['reward_to_risk'] = np.inf
    else:
      PORTFOLIO_stats['reward_to_risk'] = 0
    
    PORTFOLIO_stats['expectency_frac'] = PORTFOLIO_stats['_trades']['return_frac'].mean()

  # Buy & Hold ROI/ Sharpe
  bh_std_per_bar = df['Close'].pct_change().std()
  PORTFOLIO_stats['bh_return_frac'] = (df['Close'][-1] - df['Close'][0]) / df['Close'][0]
  PORTFOLIO_stats['bh_return_frac_ann'] =  ((1 + PORTFOLIO_stats['bh_return_frac']) ** (365/PORTFOLIO_stats['dur'].days)) - 1
  PORTFOLIO_stats['bh_volatility_ann'] = bh_std_per_bar * (no_bar_in_yr ** 0.5)
  PORTFOLIO_stats['bh_sharpe_ratio_ann'] = (PORTFOLIO_stats['bh_return_frac_ann'] - 0) / PORTFOLIO_stats['bh_volatility_ann']

  return PORTFOLIO_stats


def report_summary(backtest, print_report = True, return_list = False):
  start = backtest['start']
  end = backtest['end']
  dur = backtest['dur']
  init_capital = round(backtest['init_capital'],2)
  final_capital = round(backtest['final_capital'],2)
  roi = round(backtest['return_frac']*100,2)
  roi_ann = round(backtest['return_frac_ann']*100,2)
  volatility_ann = round(backtest['volatility_ann']*100,2)
  sharpe_ratio_ann = round(backtest['sharpe_ratio_ann'],2)
  port_mdd = round(backtest['port_mdd_frac']*100,2)
  trade_mdd = round(backtest['trade_mdd_frac']*100,2)
  trade_P10_mdd = round(backtest['trade_P10mdd_frac']*100,2)
  win_rate = round(backtest['win_rate']*100,2)
  no_trades = backtest['no_trades']
  avg_hld_dur = round(backtest['avg_hld_dur'].total_seconds()/3600,1)
  max_hld_dur = round(backtest['max_hld_dur'].total_seconds()/3600,1)
  reward_to_risk = round(backtest['reward_to_risk'],2)
  expectency = round(backtest['expectency_frac']*100,2)


  if print_report:
    print('Start : \t\t\t' + str(start))
    print('End : \t\t\t\t' + str(end))
    print('Duration : \t\t\t' + str(dur))
    print('Initial Capital : \t\t' + str(init_capital) + ' $')
    print('Final Capital : \t\t' + str(final_capital) + ' $')
    print('ROI : \t\t\t\t' + str(roi) + ' %')
    print('Ann. ROI : \t\t\t' + str(roi_ann) + ' %')
    print('Ann. Volatility : \t\t' + str(volatility_ann) + ' %')
    print('Ann. Sharpe Ratio : \t\t' + str(sharpe_ratio_ann))
    print('Port Max. Drawdown : \t\t' + str(port_mdd) + ' %')
    print('Trade Max. Drawdown : \t\t' + str(trade_mdd) + ' %')
    print('Trade P10 Max. Drawdown : \t' + str(trade_P10_mdd) + ' %')
    print('Win Rate : \t\t\t' + str(win_rate) + ' %')
    print('No. of trades : \t\t' + str(no_trades) + ' times')
    print('Avg. Holding Duration : \t' + str(avg_hld_dur) + ' hrs.')
    print('Max. Holding Duration : \t' + str(max_hld_dur) + ' hrs.')
    print('Rewards to Risk : \t\t' + str(reward_to_risk) + ' times')
    print('Expectency : \t\t\t' + str(expectency) + ' %')
    print('='*60)
    print('Buy&Hold ROI : \t\t\t' + str(round(backtest['bh_return_frac']*100,2)) + ' %')
    print('Buy&Hold Ann. ROI : \t\t' + str(round(backtest['bh_return_frac_ann']*100,2)) + ' %')
    print('Buy&Hold Ann. Volatility : \t' + str(round(backtest['bh_volatility_ann']*100,2)) + ' %')
    print('Buy&Hold Ann. Sharpe Ratio : \t' + str(round(backtest['bh_sharpe_ratio_ann'],2)))
    print('='*60)
    print()
  
  if return_list:
    return [start, end, dur, init_capital, final_capital, roi, roi_ann, volatility_ann, sharpe_ratio_ann, 
            port_mdd, trade_mdd, trade_P10_mdd, win_rate, no_trades, avg_hld_dur, max_hld_dur, reward_to_risk, expectency]


def RSI_model_results(df, date_index, initial_capital = 100):
  df_evaluation = df[OHLC_features + targets2].copy(deep=True)
  df_evaluation['RSI'] = ta.momentum.RSIIndicator(close=df_evaluation['Close'], window=n_ind, fillna=False).rsi()
  df_evaluation = df_evaluation[test_date[0]:test_date[-1]]

  # thrs_range = np.linspace(10, 90, 9)
  # thrs_range_l_opn = thrs_range
  # thrs_range_l_cls = thrs_range
  # thrs_range_s_opn = thrs_range
  # thrs_range_s_cls = thrs_range
  thrs_range_l_opn = [30]
  thrs_range_l_cls = [70]
  thrs_range_s_opn = [70]
  thrs_range_s_cls = [30]

  # Long
  df_thrs_l = []
  for thrs_l_opn in thrs_range_l_opn:
    for thrs_l_cls in thrs_range_l_cls:
      df_evaluation_thrs = RSI_strategy(df_evaluation, thrs_l = [thrs_l_opn, thrs_l_cls], thrs_s = [np.nan, np.nan])
      backtest_thrs = Backtest(df_evaluation_thrs, df_evaluation_thrs.index[0], df_evaluation_thrs.index[-1], col_name = None, initial_capital = initial_capital, trd_fee_frac = trd_fee_frac, hld_fee_frac_hr = hld_fee_frac_hr)
      df_thrs_l.append([thrs_l_opn, thrs_l_cls, np.nan, np.nan] + report_summary(backtest_thrs, print_report = False, return_list = True))

  df_thrs_l = pd.DataFrame(data = df_thrs_l, columns=['thrs_l_opn', 'thrs_l_cls', 'thrs_s_opn', 'thrs_s_cls', 
                                                'start', 'end', 'dur', 'init_capital', 'final_capital', 'roi', 'roi_ann', 'volatility_ann', 'sharpe_ratio_ann', 
                                                'port_mdd', 'trade_mdd', 'trade_P10_mdd', 'win_rate', 'no_trades', 'avg_hld_dur', 'max_hld_dur', 'reward_to_risk', 'expectency'])

  df_thrs_l = df_thrs_l.sort_values(by=['roi'], ascending=False).reset_index(drop=True)

  # Short
  df_thrs_s = []
  for thrs_s_opn in thrs_range_s_opn:
    for thrs_s_cls in thrs_range_s_cls:
      df_evaluation_thrs = RSI_strategy(df_evaluation, thrs_l = [np.nan, np.nan], thrs_s = [thrs_s_opn, thrs_s_cls])
      backtest_thrs = Backtest(df_evaluation_thrs, df_evaluation_thrs.index[0], df_evaluation_thrs.index[-1], col_name = None, initial_capital = initial_capital, trd_fee_frac = trd_fee_frac, hld_fee_frac_hr = hld_fee_frac_hr)
      df_thrs_s.append([np.nan, np.nan, thrs_s_opn, thrs_s_cls] + report_summary(backtest_thrs, print_report = False, return_list = True))

  df_thrs_s = pd.DataFrame(data = df_thrs_s, columns=['thrs_l_opn', 'thrs_l_cls', 'thrs_s_opn', 'thrs_s_cls', 
                                                'start', 'end', 'dur', 'init_capital', 'final_capital', 'roi', 'roi_ann', 'volatility_ann', 'sharpe_ratio_ann', 
                                                'port_mdd', 'trade_mdd', 'trade_P10_mdd', 'win_rate', 'no_trades', 'avg_hld_dur', 'max_hld_dur', 'reward_to_risk', 'expectency'])


  df_thrs_s = df_thrs_s.sort_values(by=['roi'], ascending=False).reset_index(drop=True)

  # # Backtest report
  thrs_report = {'backtest_opt_l': {'thrs_l':[df_thrs_l['thrs_l_opn'][0], df_thrs_l['thrs_l_cls'][0]], 'thrs_s':[np.nan, np.nan]},
                 'backtest_opt_s': {'thrs_l':[np.nan, np.nan], 'thrs_s':[df_thrs_s['thrs_s_opn'][0], df_thrs_s['thrs_s_cls'][0]]},
                 'backtest_0_l': {'thrs_l':[30, 70], 'thrs_s':[np.nan, np.nan]}, # normal trigger point for RSI strategy = 30/70
                 'backtest_0_s': {'thrs_l':[np.nan, np.nan], 'thrs_s':[70, 30]} # normal trigger point for RSI strategy = 30/70
                }

  backtest_result = {}
  df_backtest_result = []

  for k, v in thrs_report.items():
    df_evaluation_thrs = RSI_strategy(df_evaluation, thrs_l = [v['thrs_l'][0], v['thrs_l'][1]], thrs_s = [v['thrs_s'][0], v['thrs_s'][1]])
    backtest_evaluation = Backtest(df_evaluation_thrs, df_evaluation_thrs.index[0], df_evaluation_thrs.index[-1], col_name = None, initial_capital = initial_capital, trd_fee_frac = trd_fee_frac, hld_fee_frac_hr = hld_fee_frac_hr)
    backtest_result[k] = {'df': df_evaluation_thrs.copy(deep=True),
                          'backtest': backtest_evaluation.copy()}
    df_backtest_result.append([k] + report_summary(backtest_thrs, print_report = False, return_list = True))

  df_backtest_result = pd.DataFrame(data = df_backtest_result, columns=['name', 
                                                'start', 'end', 'dur', 'init_capital', 'final_capital', 'roi', 'roi_ann', 'volatility_ann', 'sharpe_ratio_ann', 
                                                'port_mdd', 'trade_mdd', 'trade_P10_mdd', 'win_rate', 'no_trades', 'avg_hld_dur', 'max_hld_dur', 'reward_to_risk', 'expectency'])
  return df_thrs_l, df_thrs_s, backtest_result, df_backtest_result


# create strategy
def RSI_strategy(df,  thrs_l = [0, 0], # [Open (>=) , Close (<=)]
                             thrs_s = [0, 0], signal = 'RSI'):   # [Open (<=) , Close (>=)]
  df['thrs_l_opn'] = thrs_l[0]
  df['thrs_l_cls'] = thrs_l[1]
  df['thrs_s_opn'] = thrs_s[0]
  df['thrs_s_cls'] = thrs_s[1]

  df['Long_Position'] = np.nan
  df['Long_Price'] = np.nan
  df['Short_Position'] = np.nan
  df['Short_Price'] = np.nan

  for i in range(len(df)):
    idx = df.index[i]
    # long position =======
    if i == 0: 
      df.loc[idx, 'Long_Position'] = 0
      pass

    elif df[signal][i-1] < df['thrs_l_opn'][i] and df[signal][i] >= df['thrs_l_opn'][i]: # smooth return pass thrs upward
      # trigger open long position
      df.loc[idx, 'Long_Position'] = 1
      df.loc[idx, 'Long_Price'] = df['Close'][i]

    elif (df[signal][i-1] > df['thrs_l_cls'][i] and df[signal][i] <= df['thrs_l_cls'][i]) or i == len(df) - 1: # smooth return pass thrs downward
      # trigger close long position 
      df.loc[idx, 'Long_Position'] = 0
      df.loc[idx, 'Long_Price'] = df['Close'][i]
    
    else:
      # do nothing (continue last position)
      df.loc[idx, 'Long_Position'] = df['Long_Position'][i-1]


    # short position =======
    if i == 0: 
      df.loc[idx, 'Short_Position'] = 0

    elif df[signal][i-1] > df['thrs_s_opn'][i] and df[signal][i] <= df['thrs_s_opn'][i]: # smooth return pass thrs downward
      # trigger open short position
      df.loc[idx, 'Short_Position'] = 1
      df.loc[idx, 'Short_Price'] = df['Close'][i]

    elif (df[signal][i-1] < df['thrs_s_cls'][i] and df[signal][i] >= df['thrs_s_cls'][i]) or i == len(df) - 1: # smooth return pass thrs upward
      # trigger close short position 
      df.loc[idx, 'Short_Position'] = 0
      df.loc[idx, 'Short_Price'] = df['Close'][i]
    
    else:
      # do nothing (continue last position)
      df.loc[idx, 'Short_Position'] = df['Short_Position'][i-1]

  return df

def RSI_report_plot(df, backtest):
  # Plot Graph

  fig,a =  plt.subplots(3, 1, figsize=(16,9), gridspec_kw={'height_ratios': [1, 2, 1]})

  # Section 1) port curve ==========================
  lns1 = a[0].plot(backtest['_curve']['port_value'], label='Port. value')
  a[0].set_ylim(backtest['_curve']['port_value'].min()*0.8,backtest['_curve']['port_value'].max()*1.1)
  a02 = a[0].twinx()
  lns2 = a02.plot(backtest['_curve']['port_size'], label='Asset Exposure', color='r', linewidth=0.5)
  # a02.set_ylim(min(0, backtest['_curve']['port_size'].min()),backtest['_curve']['port_size'].max())
  a02.grid(False)
  lns = lns1 + lns2
  labs = [l.get_label() for l in lns]
  a[0].legend(lns, labs, bbox_to_anchor=(1.15, 1))

  # Section 2) asset price curve =======================
  df['Open_Long_Price'] = np.where(df['Long_Position'].shift(periods=1) < df['Long_Position'], df['Long_Price'], np.nan)
  df['Close_Long_Price'] = np.where(df['Long_Position'].shift(periods=1) > df['Long_Position'], df['Long_Price'], np.nan)
  df['Open_Short_Price'] = np.where(df['Short_Position'].shift(periods=1) < df['Short_Position'], df['Short_Price'], np.nan)
  df['Close_Short_Price'] = np.where(df['Short_Position'].shift(periods=1) > df['Short_Position'], df['Short_Price'], np.nan)
  if df['Long_Position'][0] > 0: df['Open_Long_Price'][0] = df['Long_Price'][0]
  if df['Short_Position'][0] > 0: df['Open_Short_Price'][0] = df['Shprt_Price'][0] 

  a[1].plot(df['Close'], label='Actual', linewidth=0.5)
  a[1].plot(df['Smoothed_Price*'], label='Smoothed_Price*')  
  
  if not df['Open_Long_Price'].isnull().all():
    a[1].scatter(df.index, df['Open_Long_Price'], c='green', label='Open_Long_Price')  
  if not df['Close_Long_Price'].isnull().all():
    a[1].scatter(df.index, df['Close_Long_Price'], c='red', label='Close_Long_Price')  
  if not df['Open_Short_Price'].isnull().all():
    a[1].scatter(df.index, df['Open_Short_Price'], c='red', label='Open_Short_Price', marker='x')  
  if not df['Close_Short_Price'].isnull().all():
    a[1].scatter(df.index, df['Close_Short_Price'], c='green', label='Close_Short_Price', marker='x')

  a[1].legend(bbox_to_anchor=(1.15, 1))
  a[1].set_ylim(df['Low'].min()*0.9,df['High'].max()*1.1)

  # Section 3) indicator curve =============================
  a[2].plot(df['RSI'], label='RSI')
  if not df['thrs_l_opn'].isnull().all():
    a[2].plot(df['thrs_l_opn'], label='thrs_l_opn', linewidth=0.5, c='green')
  if not df['thrs_l_cls'].isnull().all():
    a[2].plot(df['thrs_l_cls'], label='thrs_l_cls', linewidth=0.5, c='red')
  if not df['thrs_s_opn'].isnull().all():
    a[2].plot(df['thrs_s_opn'], label='thrs_s_opn', linewidth=0.5, c='red', linestyle=(0, (10,3)))
  if not df['thrs_s_cls'].isnull().all():
    a[2].plot(df['thrs_s_cls'], label='thrs_s_cls', linewidth=0.5, c='green', linestyle=(0, (10,3)))
  a[2].legend(bbox_to_anchor=(1, 1))

  plt.show()
  print()

def RSI_report_summary_plot(df, backtest):
  report_summary(backtest, print_report = True, return_list = False)
  RSI_report_plot(df, backtest)

def BH_model_results(df, date_index, initial_capital = 100):
  df_evaluation = df[OHLC_features + targets2].copy(deep=True)
  df_evaluation = df_evaluation[test_date[0]:test_date[-1]]

  # Long
  df_thrs_l = []
  df_evaluation_thrs = BH_strategy(df_evaluation, position = ['long'])
  backtest_thrs = Backtest(df_evaluation_thrs, df_evaluation_thrs.index[0], df_evaluation_thrs.index[-1], col_name = None, initial_capital = initial_capital, trd_fee_frac = trd_fee_frac, hld_fee_frac_hr = hld_fee_frac_hr)
  df_thrs_l.append([np.nan, np.nan, np.nan, np.nan] + report_summary(backtest_thrs, print_report = False, return_list = True))

  df_thrs_l = pd.DataFrame(data = df_thrs_l, columns=['thrs_l_opn', 'thrs_l_cls', 'thrs_s_opn', 'thrs_s_cls', 
                                                'start', 'end', 'dur', 'init_capital', 'final_capital', 'roi', 'roi_ann', 'volatility_ann', 'sharpe_ratio_ann', 
                                                'port_mdd', 'trade_mdd', 'trade_P10_mdd', 'win_rate', 'no_trades', 'avg_hld_dur', 'max_hld_dur', 'reward_to_risk', 'expectency'])

  df_thrs_l = df_thrs_l.sort_values(by=['roi'], ascending=False).reset_index(drop=True)

  # Short
  df_thrs_s = []
  df_evaluation_thrs = BH_strategy(df_evaluation, position = ['short'])
  backtest_thrs = Backtest(df_evaluation_thrs, df_evaluation_thrs.index[0], df_evaluation_thrs.index[-1], col_name = None, initial_capital = initial_capital, trd_fee_frac = trd_fee_frac, hld_fee_frac_hr = hld_fee_frac_hr)
  df_thrs_s.append([np.nan, np.nan, np.nan, np.nan] + report_summary(backtest_thrs, print_report = False, return_list = True))

  df_thrs_s = pd.DataFrame(data = df_thrs_s, columns=['thrs_l_opn', 'thrs_l_cls', 'thrs_s_opn', 'thrs_s_cls', 
                                                'start', 'end', 'dur', 'init_capital', 'final_capital', 'roi', 'roi_ann', 'volatility_ann', 'sharpe_ratio_ann', 
                                                'port_mdd', 'trade_mdd', 'trade_P10_mdd', 'win_rate', 'no_trades', 'avg_hld_dur', 'max_hld_dur', 'reward_to_risk', 'expectency'])


  df_thrs_s = df_thrs_s.sort_values(by=['roi'], ascending=False).reset_index(drop=True)

  # Backtest report
  thrs_report = {'backtest_opt_l': {'thrs_l':[df_thrs_l['thrs_l_opn'][0], df_thrs_l['thrs_l_cls'][0]], 'thrs_s':[np.nan, np.nan]},
                 'backtest_opt_s': {'thrs_l':[np.nan, np.nan], 'thrs_s':[df_thrs_s['thrs_s_opn'][0], df_thrs_s['thrs_s_cls'][0]]},
                 'backtest_0_l': {'thrs_l':[np.nan, np.nan], 'thrs_s':[np.nan, np.nan]},
                 'backtest_0_s': {'thrs_l':[np.nan, np.nan], 'thrs_s':[np.nan, np.nan]}
                }

  backtest_result = {}
  df_backtest_result = []

  for k, v in thrs_report.items():
    if k in ['backtest_opt_l', 'backtest_0_l']:
      position = ['long']
    elif k in ['backtest_opt_s', 'backtest_0_s']:
      position = ['short']
    
    df_evaluation_thrs = BH_strategy(df_evaluation, position = position)
    backtest_evaluation = Backtest(df_evaluation_thrs, df_evaluation_thrs.index[0], df_evaluation_thrs.index[-1], col_name = None, initial_capital = initial_capital, trd_fee_frac = trd_fee_frac, hld_fee_frac_hr = hld_fee_frac_hr)
    backtest_result[k] = {'df': df_evaluation_thrs.copy(deep=True),
                          'backtest': backtest_evaluation.copy()}
    df_backtest_result.append([k] + report_summary(backtest_thrs, print_report = False, return_list = True))

  df_backtest_result = pd.DataFrame(data = df_backtest_result, columns=['name', 
                                                'start', 'end', 'dur', 'init_capital', 'final_capital', 'roi', 'roi_ann', 'volatility_ann', 'sharpe_ratio_ann', 
                                                'port_mdd', 'trade_mdd', 'trade_P10_mdd', 'win_rate', 'no_trades', 'avg_hld_dur', 'max_hld_dur', 'reward_to_risk', 'expectency'])
  return df_thrs_l, df_thrs_s, backtest_result, df_backtest_result


# create strategy
def BH_strategy(df, position = ['long']): # position = ['long'], ['short'], ['long', 'short']

  df['Long_Position'] = np.nan
  df['Long_Price'] = np.nan
  df['Short_Position'] = np.nan
  df['Short_Price'] = np.nan

  for i in range(len(df)):
    idx = df.index[i]
    # long position =======
    if 'long' in position:
      if i == 0: 
        # trigger open long position
        df.loc[idx, 'Long_Position'] = 1
        df.loc[idx, 'Long_Price'] = df['Close'][i]
      
      elif i == len(df) - 1:
        # trigger close long position 
        df.loc[idx, 'Long_Position'] = 0
        df.loc[idx, 'Long_Price'] = df['Close'][i]
      
      else:
        # do nothing (continue last position)
        df.loc[idx, 'Long_Position'] = df['Long_Position'][i-1]


    # short position =======
    if 'short' in position:
      if i == 0: 
        # trigger open short position
        df.loc[idx, 'Short_Position'] = 1
        df.loc[idx, 'Short_Price'] = df['Close'][i]
      
      elif i == len(df) - 1:
        # trigger close short position 
        df.loc[idx, 'Short_Position'] = 0
        df.loc[idx, 'Short_Price'] = df['Close'][i]

      else:
        # do nothing (continue last position)
        df.loc[idx, 'Short_Position'] = df['Short_Position'][i-1]

  return df

def BH_report_plot(df, backtest):
  # Plot Graph

  fig,a =  plt.subplots(2, 1, figsize=(16,9), gridspec_kw={'height_ratios': [1, 2]})

  # Section 1) port curve ==========================
  lns1 = a[0].plot(backtest['_curve']['port_value'], label='Port. value')
  a[0].set_ylim(backtest['_curve']['port_value'].min()*0.8,backtest['_curve']['port_value'].max()*1.1)
  a02 = a[0].twinx()
  lns2 = a02.plot(backtest['_curve']['port_size'], label='Asset Exposure', color='r', linewidth=0.5)
  # a02.set_ylim(min(0, backtest['_curve']['port_size'].min()),backtest['_curve']['port_size'].max())
  a02.grid(False)
  lns = lns1 + lns2
  labs = [l.get_label() for l in lns]
  a[0].legend(lns, labs, bbox_to_anchor=(1.15, 1))

  # Section 2) asset price curve =======================
  df['Open_Long_Price'] = np.where(df['Long_Position'].shift(periods=1) < df['Long_Position'], df['Long_Price'], np.nan)
  df['Close_Long_Price'] = np.where(df['Long_Position'].shift(periods=1) > df['Long_Position'], df['Long_Price'], np.nan)
  df['Open_Short_Price'] = np.where(df['Short_Position'].shift(periods=1) < df['Short_Position'], df['Short_Price'], np.nan)
  df['Close_Short_Price'] = np.where(df['Short_Position'].shift(periods=1) > df['Short_Position'], df['Short_Price'], np.nan)
  if df['Long_Position'][0] > 0: df['Open_Long_Price'][0] = df['Long_Price'][0]
  if df['Short_Position'][0] > 0: df['Open_Short_Price'][0] = df['Shprt_Price'][0] 

  a[1].plot(df['Close'], label='Actual', linewidth=0.5)
  a[1].plot(df['Smoothed_Price*'], label='Smoothed_Price*')  
  
  if not df['Open_Long_Price'].isnull().all():
    a[1].scatter(df.index, df['Open_Long_Price'], c='green', label='Open_Long_Price')  
  if not df['Close_Long_Price'].isnull().all():
    a[1].scatter(df.index, df['Close_Long_Price'], c='red', label='Close_Long_Price')  
  if not df['Open_Short_Price'].isnull().all():
    a[1].scatter(df.index, df['Open_Short_Price'], c='red', label='Open_Short_Price', marker='x')  
  if not df['Close_Short_Price'].isnull().all():
    a[1].scatter(df.index, df['Close_Short_Price'], c='green', label='Close_Short_Price', marker='x')

  a[1].legend(bbox_to_anchor=(1.15, 1))
  a[1].set_ylim(df['Low'].min()*0.9,df['High'].max()*1.1)

  # # Section 3) indicator curve =============================
  # a[2].plot(df['MACD'], label='MACD')
  # a[2].plot(df['MACD_signal'], label='MACD_signal')
  # a[2].legend(bbox_to_anchor=(1, 1))

  plt.show()
  print()

def BH_report_summary_plot(df, backtest):
  report_summary(backtest, print_report = True, return_list = False)
  BH_report_plot(df, backtest)

def MACD_model_results(df, date_index, initial_capital = 100):
  df_evaluation = df[OHLC_features + targets2].copy(deep=True)
  df_evaluation['MACD'] = ta.trend.MACD(close=df['Close'], window_slow=26, window_fast=12, window_sign=9, fillna=False).macd()
  df_evaluation['MACD_signal'] = ta.trend.MACD(close=df['Close'], window_slow=26, window_fast=12, window_sign=9, fillna=False).macd_signal()
  df_evaluation = df_evaluation[test_date[0]:test_date[-1]]

  # Long
  df_thrs_l = []
  df_evaluation_thrs = MACD_strategy(df_evaluation, position = ['long'])
  backtest_thrs = Backtest(df_evaluation_thrs, df_evaluation_thrs.index[0], df_evaluation_thrs.index[-1], col_name = None, initial_capital = initial_capital, trd_fee_frac = trd_fee_frac, hld_fee_frac_hr = hld_fee_frac_hr)
  df_thrs_l.append([np.nan, np.nan, np.nan, np.nan] + report_summary(backtest_thrs, print_report = False, return_list = True))

  df_thrs_l = pd.DataFrame(data = df_thrs_l, columns=['thrs_l_opn', 'thrs_l_cls', 'thrs_s_opn', 'thrs_s_cls', 
                                                'start', 'end', 'dur', 'init_capital', 'final_capital', 'roi', 'roi_ann', 'volatility_ann', 'sharpe_ratio_ann', 
                                                'port_mdd', 'trade_mdd', 'trade_P10_mdd', 'win_rate', 'no_trades', 'avg_hld_dur', 'max_hld_dur', 'reward_to_risk', 'expectency'])

  df_thrs_l = df_thrs_l.sort_values(by=['roi'], ascending=False).reset_index(drop=True)

  # Short
  df_thrs_s = []
  df_evaluation_thrs = MACD_strategy(df_evaluation, position = ['short'])
  backtest_thrs = Backtest(df_evaluation_thrs, df_evaluation_thrs.index[0], df_evaluation_thrs.index[-1], col_name = None, initial_capital = initial_capital, trd_fee_frac = trd_fee_frac, hld_fee_frac_hr = hld_fee_frac_hr)
  df_thrs_s.append([np.nan, np.nan, np.nan, np.nan] + report_summary(backtest_thrs, print_report = False, return_list = True))

  df_thrs_s = pd.DataFrame(data = df_thrs_s, columns=['thrs_l_opn', 'thrs_l_cls', 'thrs_s_opn', 'thrs_s_cls', 
                                                'start', 'end', 'dur', 'init_capital', 'final_capital', 'roi', 'roi_ann', 'volatility_ann', 'sharpe_ratio_ann', 
                                                'port_mdd', 'trade_mdd', 'trade_P10_mdd', 'win_rate', 'no_trades', 'avg_hld_dur', 'max_hld_dur', 'reward_to_risk', 'expectency'])


  df_thrs_s = df_thrs_s.sort_values(by=['roi'], ascending=False).reset_index(drop=True)

  # Backtest report
  thrs_report = {'backtest_opt_l': {'thrs_l':[df_thrs_l['thrs_l_opn'][0], df_thrs_l['thrs_l_cls'][0]], 'thrs_s':[np.nan, np.nan]},
                 'backtest_opt_s': {'thrs_l':[np.nan, np.nan], 'thrs_s':[df_thrs_s['thrs_s_opn'][0], df_thrs_s['thrs_s_cls'][0]]},
                 'backtest_0_l': {'thrs_l':[np.nan, np.nan], 'thrs_s':[np.nan, np.nan]},
                 'backtest_0_s': {'thrs_l':[np.nan, np.nan], 'thrs_s':[np.nan, np.nan]}
                }

  backtest_result = {}
  df_backtest_result = []

  for k, v in thrs_report.items():
    if k in ['backtest_opt_l', 'backtest_0_l']:
      position = ['long']
    elif k in ['backtest_opt_s', 'backtest_0_s']:
      position = ['short']
    
    df_evaluation_thrs = MACD_strategy(df_evaluation, position = position)
    backtest_evaluation = Backtest(df_evaluation_thrs, df_evaluation_thrs.index[0], df_evaluation_thrs.index[-1], col_name = None, initial_capital = initial_capital, trd_fee_frac = trd_fee_frac, hld_fee_frac_hr = hld_fee_frac_hr)
    backtest_result[k] = {'df': df_evaluation_thrs.copy(deep=True),
                          'backtest': backtest_evaluation.copy()}
    df_backtest_result.append([k] + report_summary(backtest_thrs, print_report = False, return_list = True))

  df_backtest_result = pd.DataFrame(data = df_backtest_result, columns=['name', 
                                                'start', 'end', 'dur', 'init_capital', 'final_capital', 'roi', 'roi_ann', 'volatility_ann', 'sharpe_ratio_ann', 
                                                'port_mdd', 'trade_mdd', 'trade_P10_mdd', 'win_rate', 'no_trades', 'avg_hld_dur', 'max_hld_dur', 'reward_to_risk', 'expectency'])
  return df_thrs_l, df_thrs_s, backtest_result, df_backtest_result

# create strategy
def MACD_strategy(df, position = ['long'], signal = 'MACD', signal2 = 'MACD_signal'): # position = ['long'], ['short'], ['long', 'short']

  df['Long_Position'] = np.nan
  df['Long_Price'] = np.nan
  df['Short_Position'] = np.nan
  df['Short_Price'] = np.nan

  for i in range(len(df)):
    idx = df.index[i]
    # long position =======
    if 'long' in position:
      if i == 0: 
        df.loc[idx, 'Long_Position'] = 0
        pass

      elif df[signal][i-1] < df[signal2][i-1] and df[signal][i] >= df[signal2][i]: # MACD cross MACD_signal upward
        # trigger open long position
        df.loc[idx, 'Long_Position'] = 1
        df.loc[idx, 'Long_Price'] = df['Close'][i]

      elif (df[signal][i-1] > df[signal2][i-1] and df[signal][i] <= df[signal2][i]) or i == len(df) - 1: # MACD cross MACD_signal downward
        # trigger close long position 
        df.loc[idx, 'Long_Position'] = 0
        df.loc[idx, 'Long_Price'] = df['Close'][i]
      
      else:
        # do nothing (continue last position)
        df.loc[idx, 'Long_Position'] = df['Long_Position'][i-1]


    # short position =======
    if 'short' in position:
      if i == 0: 
        df.loc[idx, 'Short_Position'] = 0

      elif df[signal][i-1] > df[signal2][i-1] and df[signal][i] <= df[signal2][i]: # MACD cross MACD_signal downward
        # trigger open short position
        df.loc[idx, 'Short_Position'] = 1
        df.loc[idx, 'Short_Price'] = df['Close'][i]

      elif (df[signal][i-1] < df[signal2][i-1] and df[signal][i] >= df[signal2][i]) or i == len(df) - 1: # MACD cross MACD_signal upward
        # trigger close short position 
        df.loc[idx, 'Short_Position'] = 0
        df.loc[idx, 'Short_Price'] = df['Close'][i]
      
      else:
        # do nothing (continue last position)
        df.loc[idx, 'Short_Position'] = df['Short_Position'][i-1]

  return df

def MACD_report_plot(df, backtest):
  # Plot Graph

  fig,a =  plt.subplots(3, 1, figsize=(16,9), gridspec_kw={'height_ratios': [1, 2, 1]})

  # Section 1) port curve ==========================
  lns1 = a[0].plot(backtest['_curve']['port_value'], label='Port. value')
  a[0].set_ylim(backtest['_curve']['port_value'].min()*0.8,backtest['_curve']['port_value'].max()*1.1)
  a02 = a[0].twinx()
  lns2 = a02.plot(backtest['_curve']['port_size'], label='Asset Exposure', color='r', linewidth=0.5)
  # a02.set_ylim(min(0, backtest['_curve']['port_size'].min()),backtest['_curve']['port_size'].max())
  a02.grid(False)
  lns = lns1 + lns2
  labs = [l.get_label() for l in lns]
  a[0].legend(lns, labs, bbox_to_anchor=(1.15, 1))

  # Section 2) asset price curve =======================
  df['Open_Long_Price'] = np.where(df['Long_Position'].shift(periods=1) < df['Long_Position'], df['Long_Price'], np.nan)
  df['Close_Long_Price'] = np.where(df['Long_Position'].shift(periods=1) > df['Long_Position'], df['Long_Price'], np.nan)
  df['Open_Short_Price'] = np.where(df['Short_Position'].shift(periods=1) < df['Short_Position'], df['Short_Price'], np.nan)
  df['Close_Short_Price'] = np.where(df['Short_Position'].shift(periods=1) > df['Short_Position'], df['Short_Price'], np.nan)
  if df['Long_Position'][0] > 0: df['Open_Long_Price'][0] = df['Long_Price'][0]
  if df['Short_Position'][0] > 0: df['Open_Short_Price'][0] = df['Shprt_Price'][0] 

  a[1].plot(df['Close'], label='Actual', linewidth=0.5)
  a[1].plot(df['Smoothed_Price*'], label='Smoothed_Price*')  
  
  if not df['Open_Long_Price'].isnull().all():
    a[1].scatter(df.index, df['Open_Long_Price'], c='green', label='Open_Long_Price')  
  if not df['Close_Long_Price'].isnull().all():
    a[1].scatter(df.index, df['Close_Long_Price'], c='red', label='Close_Long_Price')  
  if not df['Open_Short_Price'].isnull().all():
    a[1].scatter(df.index, df['Open_Short_Price'], c='red', label='Open_Short_Price', marker='x')  
  if not df['Close_Short_Price'].isnull().all():
    a[1].scatter(df.index, df['Close_Short_Price'], c='green', label='Close_Short_Price', marker='x')

  a[1].legend(bbox_to_anchor=(1.15, 1))
  a[1].set_ylim(df['Low'].min()*0.9,df['High'].max()*1.1)

  # Section 3) indicator curve =============================
  a[2].plot(df['MACD'], label='MACD')
  a[2].plot(df['MACD_signal'], label='MACD_signal')
  a[2].legend(bbox_to_anchor=(1, 1))

  plt.show()
  print()

def MACD_report_summary_plot(df, backtest):
  report_summary(backtest, print_report = True, return_list = False)
  MACD_report_plot(df, backtest)

In [ ]:
symbols = 'CL=F'
# Available kline size "1m", "5m", "15m", "1h", "4h", "1d" for binance
kline_size = '30m'
trd_fee_pct = 0.0 # trading fee in percent
trd_fee_frac = trd_fee_pct/100
hld_fee_pct_8hr = 0 # holding fee in percent / 8hr (normal rate in Binance Future = 0.02%)
hld_fee_frac_hr = hld_fee_pct_8hr/(8*100)
OHLC_features = ['Open', 'High', 'Low', 'Close']
price_features = ['Close', 'Volume', 'Rel_Open', 'Rel_High', 'Rel_Low']
ind_features = ['Rel_SMA_fast', 'Rel_SMA_slow', 'MACD', 'ADX', 'CCI', 'ROC', 'RSI', 'TSI', '%K', '%D', '%R', 'Rel_BB_hband', 'Rel_BB_lband', 'Rel_ATR', 'UI',   'CMF', 'FI', 'MFI', 'VPT', 'VWAP']
#'ADI', 'OBV' ,'MFI', 'VPT', 'VWAP'
# time_features = ['Hour_of_Day', 'Day_of_Week', 'Day_of_Month', 'Day_of_Year']
time_features = ['Day_of_Week', 'Day_of_Month', 'Day_of_Year']
trans_time_features = [f(x) for x in time_features for f in (lambda x: x + '_Sin',lambda x: x + '_Cos')]
features = price_features + trans_time_features + ind_features
targets_scale = ['label']
targets_not_scale = []
targets = targets_scale + targets_not_scale
targets2 = ['label_1',
            ] # usd only for evaluation ploting
rolling_indicator = False

# model_no: 1 = CNN_LSTM, 2 = CNN, 3 = LSTM, 4 = LSTM_CNN, 5 = Transformer
model_no = 4
test_year = 2018
case_no = 1
cnn_no_node_list = [64, 128]
lstm_no_node_list = [64, 128]
model_name_dict = {1:'CNN-LSTM', 2:'CNN', 3:'LSTM', 4:'LSTM-CNN', 5:'Transformer'}
time_beg_dict = {2015:'2005-01-01 00:00:00', 2016:'2006-01-01 00:00:00', 2017:'2007-01-01 00:00:00', 2018:'2008-01-01 00:00:00', 2019:'2009-01-01 00:00:00', 2020:'2010-01-01 00:00:00' }
time_end_dict = {2015:'2016-01-01 00:00:00', 2016:'2017-01-01 00:00:00', 2017:'2018-01-01 00:00:00', 2018:'2019-01-01 00:00:00', 2019:'2020-01-01 00:00:00', 2020:'2021-01-01 00:00:00' }
val_beg_dict = {2015:'2014-01-01 00:00:00', 2016:'2015-01-01 00:00:00', 2017:'2016-01-01 00:00:00', 2018:'2017-01-01 00:00:00', 2019:'2018-01-01 00:00:00', 2020:'2019-01-01 00:00:00' }
test_beg_dict = {2015:'2015-01-01 00:00:00', 2016:'2016-01-01 00:00:00', 2017:'2017-01-01 00:00:00', 2018:'2018-01-01 00:00:00', 2019:'2019-01-01 00:00:00', 2020:'2020-01-01 00:00:00' }


# path
data_path = '/content/gdrive/MyDrive/2022/Data/'
model_path = '/content/gdrive/MyDrive/2022/Model/'
model_filename = model_path + symbols + '-' + kline_size + '/' + str(test_year) + '/' + str(case_no) + '/' + model_name_dict[model_no]
variable_path = model_filename + '/pre-process-variables/'
variable_filename = variable_path + 'objs.pkl'


# Train / Val / Test Split ======================================
split_type = 'time' # 'fraction', 'time'
# time_beg = '2007-01-01 00:00:00' # time begin for train-val-test
# time_end = '2018-01-01 00:00:00' # time end for train-val-test
time_beg = time_beg_dict[test_year]
time_end = time_end_dict[test_year]
# for split_type = 'fraction'
val_fraction = 0.1
test_fraction = 0.1
train_fraction = 1 - val_fraction - test_fraction
# for split_type = 'time'
val_beg = val_beg_dict[test_year]
test_beg = test_beg_dict[test_year]

T =32 # T-days window of input data for predicting target_class
n_ind = 10 # indicator fast
k_ind = 15 # indicator slow
start_index_delta = - T - k_ind - 23 # import more bar befor time_beg for labeling
end_index_delta = 2 # import more bar after time_end for labeling

# Pre-procession features
pct_change_transform_features = ['Close', 'Volume', ] # non-stationary feasture that need to transform to stationary (transform be percent change from previous time-step)
quantile_transform_features = ['Volume', 'VPT'] # Transform feature with obviuos outliner
# quantile_transform_features = ['Volume']


train_model_ = True
features_plotting = False
save_model_ = True
load_model_ = not train_model_

## - SMA

In [ ]:
def SMA_model_results(df, date_index, initial_capital = 100):
  df_evaluation = df[OHLC_features + targets2].copy(deep=True)
  df_evaluation['SMA_15'] = ta.trend.SMAIndicator(close=df['Close'], window=15, fillna=False).sma_indicator()
  df_evaluation['SMA_50'] = ta.trend.SMAIndicator(close=df['Close'], window=50, fillna=False).sma_indicator()
  df_evaluation = df_evaluation[test_date[0]:test_date[-1]]

  # Long
  df_thrs_l = []
  df_evaluation_thrs = SMA_strategy(df_evaluation, position = ['long'])
  backtest_thrs = Backtest(df_evaluation_thrs, df_evaluation_thrs.index[0], df_evaluation_thrs.index[-1], col_name = None, initial_capital = initial_capital, trd_fee_frac = trd_fee_frac, hld_fee_frac_hr = hld_fee_frac_hr)
  df_thrs_l.append([np.nan, np.nan, np.nan, np.nan] + report_summary(backtest_thrs, print_report = False, return_list = True))

  df_thrs_l = pd.DataFrame(data = df_thrs_l, columns=['thrs_l_opn', 'thrs_l_cls', 'thrs_s_opn', 'thrs_s_cls', 
                                                'start', 'end', 'dur', 'init_capital', 'final_capital', 'roi', 'roi_ann', 'volatility_ann', 'sharpe_ratio_ann', 
                                                'port_mdd', 'trade_mdd', 'trade_P10_mdd', 'win_rate', 'no_trades', 'avg_hld_dur', 'max_hld_dur', 'reward_to_risk', 'expectency'])

  df_thrs_l = df_thrs_l.sort_values(by=['roi'], ascending=False).reset_index(drop=True)

  # Short
  df_thrs_s = []
  df_evaluation_thrs = SMA_strategy(df_evaluation, position = ['short'])
  backtest_thrs = Backtest(df_evaluation_thrs, df_evaluation_thrs.index[0], df_evaluation_thrs.index[-1], col_name = None, initial_capital = initial_capital, trd_fee_frac = trd_fee_frac, hld_fee_frac_hr = hld_fee_frac_hr)
  df_thrs_s.append([np.nan, np.nan, np.nan, np.nan] + report_summary(backtest_thrs, print_report = False, return_list = True))

  df_thrs_s = pd.DataFrame(data = df_thrs_s, columns=['thrs_l_opn', 'thrs_l_cls', 'thrs_s_opn', 'thrs_s_cls', 
                                                'start', 'end', 'dur', 'init_capital', 'final_capital', 'roi', 'roi_ann', 'volatility_ann', 'sharpe_ratio_ann', 
                                                'port_mdd', 'trade_mdd', 'trade_P10_mdd', 'win_rate', 'no_trades', 'avg_hld_dur', 'max_hld_dur', 'reward_to_risk', 'expectency'])


  df_thrs_s = df_thrs_s.sort_values(by=['roi'], ascending=False).reset_index(drop=True)

  # Backtest report
  thrs_report = {'backtest_opt_l': {'thrs_l':[df_thrs_l['thrs_l_opn'][0], df_thrs_l['thrs_l_cls'][0]], 'thrs_s':[np.nan, np.nan]},
                 'backtest_opt_s': {'thrs_l':[np.nan, np.nan], 'thrs_s':[df_thrs_s['thrs_s_opn'][0], df_thrs_s['thrs_s_cls'][0]]},
                 'backtest_0_l': {'thrs_l':[np.nan, np.nan], 'thrs_s':[np.nan, np.nan]},
                 'backtest_0_s': {'thrs_l':[np.nan, np.nan], 'thrs_s':[np.nan, np.nan]}
                }

  backtest_result = {}
  df_backtest_result = []

  for k, v in thrs_report.items():
    if k in ['backtest_opt_l', 'backtest_0_l']:
      position = ['long']
    elif k in ['backtest_opt_s', 'backtest_0_s']:
      position = ['short']
    
    df_evaluation_thrs = SMA_strategy(df_evaluation, position = position)
    backtest_evaluation = Backtest(df_evaluation_thrs, df_evaluation_thrs.index[0], df_evaluation_thrs.index[-1], col_name = None, initial_capital = initial_capital, trd_fee_frac = trd_fee_frac, hld_fee_frac_hr = hld_fee_frac_hr)
    backtest_result[k] = {'df': df_evaluation_thrs.copy(deep=True),
                          'backtest': backtest_evaluation.copy()}
    df_backtest_result.append([k] + report_summary(backtest_thrs, print_report = False, return_list = True))

  df_backtest_result = pd.DataFrame(data = df_backtest_result, columns=['name', 
                                                'start', 'end', 'dur', 'init_capital', 'final_capital', 'roi', 'roi_ann', 'volatility_ann', 'sharpe_ratio_ann', 
                                                'port_mdd', 'trade_mdd', 'trade_P10_mdd', 'win_rate', 'no_trades', 'avg_hld_dur', 'max_hld_dur', 'reward_to_risk', 'expectency'])
  return df_thrs_l, df_thrs_s, backtest_result, df_backtest_result

In [ ]:
# create strategy
def SMA_strategy(df, position = ['long'], signal = 'SMA_15', signal2 = 'SMA_50'): # position = ['long'], ['short'], ['long', 'short']

  df['Long_Position'] = np.nan
  df['Long_Price'] = np.nan
  df['Short_Position'] = np.nan
  df['Short_Price'] = np.nan

  for i in range(len(df)):
    idx = df.index[i]
    # long position =======
    if 'long' in position:
      if i == 0: 
        df.loc[idx, 'Long_Position'] = 0
        pass

      elif df[signal][i-1] < df[signal2][i-1] and df[signal][i] >= df[signal2][i]: # SMA50 cross SMA200 upward
        # trigger open long position
        df.loc[idx, 'Long_Position'] = 1
        df.loc[idx, 'Long_Price'] = df['Close'][i]

      elif (df[signal][i-1] > df[signal2][i-1] and df[signal][i] <= df[signal2][i]) or i == len(df) - 1: # SMA50 cross SMA200 downward
        # trigger close long position 
        df.loc[idx, 'Long_Position'] = 0
        df.loc[idx, 'Long_Price'] = df['Close'][i]
      
      else:
        # do nothing (continue last position)
        df.loc[idx, 'Long_Position'] = df['Long_Position'][i-1]


    # short position =======
    if 'short' in position:
      if i == 0: 
        df.loc[idx, 'Short_Position'] = 0

      elif df[signal][i-1] > df[signal2][i-1] and df[signal][i] <= df[signal2][i]: # SMA50 cross SMA200 downward
        # trigger open short position
        df.loc[idx, 'Short_Position'] = 1
        df.loc[idx, 'Short_Price'] = df['Close'][i]

      elif (df[signal][i-1] < df[signal2][i-1] and df[signal][i] >= df[signal2][i]) or i == len(df) - 1: # SMA50 cross SMA200 upward
        # trigger close short position 
        df.loc[idx, 'Short_Position'] = 0
        df.loc[idx, 'Short_Price'] = df['Close'][i]
      
      else:
        # do nothing (continue last position)
        df.loc[idx, 'Short_Position'] = df['Short_Position'][i-1]

  return df

In [ ]:
def SMA_report_plot(df, backtest):
  # Plot Graph

  fig,a =  plt.subplots(3, 1, figsize=(16,9), gridspec_kw={'height_ratios': [1, 2, 1]})

  # Section 1) port curve ==========================
  lns1 = a[0].plot(backtest['_curve']['port_value'], label='Port. value')
  a[0].set_ylim(backtest['_curve']['port_value'].min()*0.8,backtest['_curve']['port_value'].max()*1.1)
  a02 = a[0].twinx()
  lns2 = a02.plot(backtest['_curve']['port_size'], label='Asset Exposure', color='r', linewidth=0.5)
  # a02.set_ylim(min(0, backtest['_curve']['port_size'].min()),backtest['_curve']['port_size'].max())
  a02.grid(False)
  lns = lns1 + lns2
  labs = [l.get_label() for l in lns]
  a[0].legend(lns, labs, bbox_to_anchor=(1.15, 1))

  # Section 2) asset price curve =======================
  df['Open_Long_Price'] = np.where(df['Long_Position'].shift(periods=1) < df['Long_Position'], df['Long_Price'], np.nan)
  df['Close_Long_Price'] = np.where(df['Long_Position'].shift(periods=1) > df['Long_Position'], df['Long_Price'], np.nan)
  df['Open_Short_Price'] = np.where(df['Short_Position'].shift(periods=1) < df['Short_Position'], df['Short_Price'], np.nan)
  df['Close_Short_Price'] = np.where(df['Short_Position'].shift(periods=1) > df['Short_Position'], df['Short_Price'], np.nan)
  if df['Long_Position'][0] > 0: df['Open_Long_Price'][0] = df['Long_Price'][0]
  if df['Short_Position'][0] > 0: df['Open_Short_Price'][0] = df['Shprt_Price'][0] 

  a[1].plot(df['Close'], label='Actual', linewidth=0.5)
  a[1].plot(df['Smoothed_Price*'], label='Smoothed_Price*')  
  
  if not df['Open_Long_Price'].isnull().all():
    a[1].scatter(df.index, df['Open_Long_Price'], c='green', label='Open_Long_Price')  
  if not df['Close_Long_Price'].isnull().all():
    a[1].scatter(df.index, df['Close_Long_Price'], c='red', label='Close_Long_Price')  
  if not df['Open_Short_Price'].isnull().all():
    a[1].scatter(df.index, df['Open_Short_Price'], c='red', label='Open_Short_Price', marker='x')  
  if not df['Close_Short_Price'].isnull().all():
    a[1].scatter(df.index, df['Close_Short_Price'], c='green', label='Close_Short_Price', marker='x')

  a[1].legend(bbox_to_anchor=(1.15, 1))
  a[1].set_ylim(df['Low'].min()*0.9,df['High'].max()*1.1)

  # Section 3) indicator curve =============================
  a[2].plot(df['SMA_15'], label='SMA_15')
  a[2].plot(df['SMA_50'], label='SMA_50')
  a[2].legend(bbox_to_anchor=(1, 1))

  plt.show()
  print()

In [ ]:
def SMA_report_summary_plot(df, backtest):
  report_summary(backtest, print_report = True, return_list = False)
  SMA_report_plot(df, backtest)

## - Stochastics

In [ ]:
def STO_model_results(df, date_index, initial_capital = 100):
  df_evaluation = df[OHLC_features + targets2].copy(deep=True)
  df_evaluation['%K'] = ta.momentum.StochRSIIndicator(close=df['Close'], window=n_ind, smooth1=n_ind, smooth2=n_ind, fillna=False).stochrsi_k()*100
  df_evaluation['%D'] = ta.momentum.StochRSIIndicator(close=df['Close'], window=n_ind, smooth1=n_ind, smooth2=n_ind, fillna=False).stochrsi_d()*100
  df_evaluation = df_evaluation[test_date[0]:test_date[-1]]

  # thrs_range = np.linspace(10, 90, 9)
  # thrs_range_l_opn = thrs_range
  # thrs_range_l_cls = thrs_range
  # thrs_range_s_opn = thrs_range
  # thrs_range_s_cls = thrs_range
  thrs_range_l_opn = [20]
  thrs_range_l_cls = [80]
  thrs_range_s_opn = [80]
  thrs_range_s_cls = [20]

  # Long
  df_thrs_l = []
  for thrs_l_opn in thrs_range_l_opn:
    for thrs_l_cls in thrs_range_l_cls:
      df_evaluation_thrs = STO_strategy(df_evaluation, thrs_l = [thrs_l_opn, thrs_l_cls], thrs_s = [np.nan, np.nan])
      backtest_thrs = Backtest(df_evaluation_thrs, df_evaluation_thrs.index[0], df_evaluation_thrs.index[-1], col_name = None, initial_capital = initial_capital, trd_fee_frac = trd_fee_frac, hld_fee_frac_hr = hld_fee_frac_hr)
      df_thrs_l.append([thrs_l_opn, thrs_l_cls, np.nan, np.nan] + report_summary(backtest_thrs, print_report = False, return_list = True))

  df_thrs_l = pd.DataFrame(data = df_thrs_l, columns=['thrs_l_opn', 'thrs_l_cls', 'thrs_s_opn', 'thrs_s_cls', 
                                                'start', 'end', 'dur', 'init_capital', 'final_capital', 'roi', 'roi_ann', 'volatility_ann', 'sharpe_ratio_ann', 
                                                'port_mdd', 'trade_mdd', 'trade_P10_mdd', 'win_rate', 'no_trades', 'avg_hld_dur', 'max_hld_dur', 'reward_to_risk', 'expectency'])

  df_thrs_l = df_thrs_l.sort_values(by=['roi'], ascending=False).reset_index(drop=True)

  # Short
  df_thrs_s = []
  for thrs_s_opn in thrs_range_s_opn:
    for thrs_s_cls in thrs_range_s_cls:
      df_evaluation_thrs = STO_strategy(df_evaluation, thrs_l = [np.nan, np.nan], thrs_s = [thrs_s_opn, thrs_s_cls])
      backtest_thrs = Backtest(df_evaluation_thrs, df_evaluation_thrs.index[0], df_evaluation_thrs.index[-1], col_name = None, initial_capital = initial_capital, trd_fee_frac = trd_fee_frac, hld_fee_frac_hr = hld_fee_frac_hr)
      df_thrs_s.append([np.nan, np.nan, thrs_s_opn, thrs_s_cls] + report_summary(backtest_thrs, print_report = False, return_list = True))

  df_thrs_s = pd.DataFrame(data = df_thrs_s, columns=['thrs_l_opn', 'thrs_l_cls', 'thrs_s_opn', 'thrs_s_cls', 
                                                'start', 'end', 'dur', 'init_capital', 'final_capital', 'roi', 'roi_ann', 'volatility_ann', 'sharpe_ratio_ann', 
                                                'port_mdd', 'trade_mdd', 'trade_P10_mdd', 'win_rate', 'no_trades', 'avg_hld_dur', 'max_hld_dur', 'reward_to_risk', 'expectency'])


  df_thrs_s = df_thrs_s.sort_values(by=['roi'], ascending=False).reset_index(drop=True)

  # # Backtest report
  thrs_report = {'backtest_opt_l': {'thrs_l':[df_thrs_l['thrs_l_opn'][0], df_thrs_l['thrs_l_cls'][0]], 'thrs_s':[np.nan, np.nan]},
                 'backtest_opt_s': {'thrs_l':[np.nan, np.nan], 'thrs_s':[df_thrs_s['thrs_s_opn'][0], df_thrs_s['thrs_s_cls'][0]]},
                 'backtest_0_l': {'thrs_l':[20, 80], 'thrs_s':[np.nan, np.nan]}, # normal trigger point for STO strategy = 20/80
                 'backtest_0_s': {'thrs_l':[np.nan, np.nan], 'thrs_s':[80, 20]} # normal trigger point for STO strategy = 20/80
                }

  backtest_result = {}
  df_backtest_result = []

  for k, v in thrs_report.items():
    df_evaluation_thrs = STO_strategy(df_evaluation, thrs_l = [v['thrs_l'][0], v['thrs_l'][1]], thrs_s = [v['thrs_s'][0], v['thrs_s'][1]])
    backtest_evaluation = Backtest(df_evaluation_thrs, df_evaluation_thrs.index[0], df_evaluation_thrs.index[-1], col_name = None, initial_capital = initial_capital, trd_fee_frac = trd_fee_frac, hld_fee_frac_hr = hld_fee_frac_hr)
    backtest_result[k] = {'df': df_evaluation_thrs.copy(deep=True),
                          'backtest': backtest_evaluation.copy()}
    df_backtest_result.append([k] + report_summary(backtest_thrs, print_report = False, return_list = True))

  df_backtest_result = pd.DataFrame(data = df_backtest_result, columns=['name', 
                                                'start', 'end', 'dur', 'init_capital', 'final_capital', 'roi', 'roi_ann', 'volatility_ann', 'sharpe_ratio_ann', 
                                                'port_mdd', 'trade_mdd', 'trade_P10_mdd', 'win_rate', 'no_trades', 'avg_hld_dur', 'max_hld_dur', 'reward_to_risk', 'expectency'])
  return df_thrs_l, df_thrs_s, backtest_result, df_backtest_result

In [ ]:
# create strategy
def STO_strategy(df,  thrs_l = [0, 0], # [Open (>=) , Close (<=)]
                             thrs_s = [0, 0], signal = '%K', signal2 = '%D'):   # [Open (<=) , Close (>=)]
  df['thrs_l_opn'] = thrs_l[0]
  df['thrs_l_cls'] = thrs_l[1]
  df['thrs_s_opn'] = thrs_s[0]
  df['thrs_s_cls'] = thrs_s[1]

  df['Long_Position'] = np.nan
  df['Long_Price'] = np.nan
  df['Short_Position'] = np.nan
  df['Short_Price'] = np.nan

  for i in range(len(df)):
    idx = df.index[i]
    # long position =======
    if i == 0: 
      df.loc[idx, 'Long_Position'] = 0
      pass

    elif df[signal2][i] <= df['thrs_l_opn'][i] and df[signal][i-1] < df[signal2][i-1] and df[signal][i] >= df[signal2][i]: # %D <= thres , %K cross %D upward
      # trigger open long position
      df.loc[idx, 'Long_Position'] = 1
      df.loc[idx, 'Long_Price'] = df['Close'][i]

    elif (df[signal2][i] >= df['thrs_l_cls'][i] and df[signal][i-1] > df[signal2][i-1] and df[signal][i] <= df[signal2][i]) or i == len(df) - 1: # %D >= thres , %K cross %D downward
      # trigger close long position 
      df.loc[idx, 'Long_Position'] = 0
      df.loc[idx, 'Long_Price'] = df['Close'][i]
    
    else:
      # do nothing (continue last position)
      df.loc[idx, 'Long_Position'] = df['Long_Position'][i-1]


    # short position =======
    if i == 0: 
      df.loc[idx, 'Short_Position'] = 0

    elif df[signal2][i] >= df['thrs_s_cls'][i] and df[signal][i-1] > df[signal2][i-1] and df[signal][i] <= df[signal2][i]: # %D >= thres , %K cross %D downward
      # trigger open short position
      df.loc[idx, 'Short_Position'] = 1
      df.loc[idx, 'Short_Price'] = df['Close'][i]

    elif (df[signal2][i] <= df['thrs_s_opn'][i] and df[signal][i-1] < df[signal2][i-1] and df[signal][i] >= df[signal2][i]) or i == len(df) - 1: # %D <= thres , %K cross %D upward
      # trigger close short position 
      df.loc[idx, 'Short_Position'] = 0
      df.loc[idx, 'Short_Price'] = df['Close'][i]
    
    else:
      # do nothing (continue last position)
      df.loc[idx, 'Short_Position'] = df['Short_Position'][i-1]

  return df

In [ ]:
def STO_report_plot(df, backtest):
  # Plot Graph

  fig,a =  plt.subplots(3, 1, figsize=(16,9), gridspec_kw={'height_ratios': [1, 2, 1]})

  # Section 1) port curve ==========================
  lns1 = a[0].plot(backtest['_curve']['port_value'], label='Port. value')
  a[0].set_ylim(backtest['_curve']['port_value'].min()*0.8,backtest['_curve']['port_value'].max()*1.1)
  a02 = a[0].twinx()
  lns2 = a02.plot(backtest['_curve']['port_size'], label='Asset Exposure', color='r', linewidth=0.5)
  # a02.set_ylim(min(0, backtest['_curve']['port_size'].min()),backtest['_curve']['port_size'].max())
  a02.grid(False)
  lns = lns1 + lns2
  labs = [l.get_label() for l in lns]
  a[0].legend(lns, labs, bbox_to_anchor=(1.15, 1))

  # Section 2) asset price curve =======================
  df['Open_Long_Price'] = np.where(df['Long_Position'].shift(periods=1) < df['Long_Position'], df['Long_Price'], np.nan)
  df['Close_Long_Price'] = np.where(df['Long_Position'].shift(periods=1) > df['Long_Position'], df['Long_Price'], np.nan)
  df['Open_Short_Price'] = np.where(df['Short_Position'].shift(periods=1) < df['Short_Position'], df['Short_Price'], np.nan)
  df['Close_Short_Price'] = np.where(df['Short_Position'].shift(periods=1) > df['Short_Position'], df['Short_Price'], np.nan)
  if df['Long_Position'][0] > 0: df['Open_Long_Price'][0] = df['Long_Price'][0]
  if df['Short_Position'][0] > 0: df['Open_Short_Price'][0] = df['Shprt_Price'][0] 

  a[1].plot(df['Close'], label='Actual', linewidth=0.5)
  a[1].plot(df['Smoothed_Price*'], label='Smoothed_Price*')  
  
  if not df['Open_Long_Price'].isnull().all():
    a[1].scatter(df.index, df['Open_Long_Price'], c='green', label='Open_Long_Price')  
  if not df['Close_Long_Price'].isnull().all():
    a[1].scatter(df.index, df['Close_Long_Price'], c='red', label='Close_Long_Price')  
  if not df['Open_Short_Price'].isnull().all():
    a[1].scatter(df.index, df['Open_Short_Price'], c='red', label='Open_Short_Price', marker='x')  
  if not df['Close_Short_Price'].isnull().all():
    a[1].scatter(df.index, df['Close_Short_Price'], c='green', label='Close_Short_Price', marker='x')

  a[1].legend(bbox_to_anchor=(1.15, 1))
  a[1].set_ylim(df['Low'].min()*0.9,df['High'].max()*1.1)

  # Section 3) indicator curve =============================
  a[2].plot(df['%K'], label='%K')
  a[2].plot(df['%D'], label='%D')
  if not df['thrs_l_opn'].isnull().all():
    a[2].plot(df['thrs_l_opn'], label='thrs_l_opn', linewidth=0.5, c='green')
  if not df['thrs_l_cls'].isnull().all():
    a[2].plot(df['thrs_l_cls'], label='thrs_l_cls', linewidth=0.5, c='red')
  if not df['thrs_s_opn'].isnull().all():
    a[2].plot(df['thrs_s_opn'], label='thrs_s_opn', linewidth=0.5, c='red', linestyle=(0, (10,3)))
  if not df['thrs_s_cls'].isnull().all():
    a[2].plot(df['thrs_s_cls'], label='thrs_s_cls', linewidth=0.5, c='green', linestyle=(0, (10,3)))
  a[2].legend(bbox_to_anchor=(1, 1))

  plt.show()
  print()

In [ ]:
def STO_report_summary_plot(df, backtest):
  report_summary(backtest, print_report = True, return_list = False)
  STO_report_plot(df, backtest)

In [ ]:
def SMA_model_results(df, date_index, initial_capital = 100):
  df_evaluation = df[OHLC_features + targets2].copy(deep=True)
  df_evaluation['SMA_15'] = ta.trend.SMAIndicator(close=df['Close'], window=15, fillna=False).sma_indicator()
  df_evaluation['SMA_50'] = ta.trend.SMAIndicator(close=df['Close'], window=50, fillna=False).sma_indicator()
  df_evaluation = df_evaluation[test_date[0]:test_date[-1]]

  # Long
  df_thrs_l = []
  df_evaluation_thrs = SMA_strategy(df_evaluation, position = ['long'])
  backtest_thrs = Backtest(df_evaluation_thrs, df_evaluation_thrs.index[0], df_evaluation_thrs.index[-1], col_name = None, initial_capital = initial_capital, trd_fee_frac = trd_fee_frac, hld_fee_frac_hr = hld_fee_frac_hr)
  df_thrs_l.append([np.nan, np.nan, np.nan, np.nan] + report_summary(backtest_thrs, print_report = False, return_list = True))

  df_thrs_l = pd.DataFrame(data = df_thrs_l, columns=['thrs_l_opn', 'thrs_l_cls', 'thrs_s_opn', 'thrs_s_cls', 
                                                'start', 'end', 'dur', 'init_capital', 'final_capital', 'roi', 'roi_ann', 'volatility_ann', 'sharpe_ratio_ann', 
                                                'port_mdd', 'trade_mdd', 'trade_P10_mdd', 'win_rate', 'no_trades', 'avg_hld_dur', 'max_hld_dur', 'reward_to_risk', 'expectency'])

  df_thrs_l = df_thrs_l.sort_values(by=['roi'], ascending=False).reset_index(drop=True)

  # Short
  df_thrs_s = []
  df_evaluation_thrs = SMA_strategy(df_evaluation, position = ['short'])
  backtest_thrs = Backtest(df_evaluation_thrs, df_evaluation_thrs.index[0], df_evaluation_thrs.index[-1], col_name = None, initial_capital = initial_capital, trd_fee_frac = trd_fee_frac, hld_fee_frac_hr = hld_fee_frac_hr)
  df_thrs_s.append([np.nan, np.nan, np.nan, np.nan] + report_summary(backtest_thrs, print_report = False, return_list = True))

  df_thrs_s = pd.DataFrame(data = df_thrs_s, columns=['thrs_l_opn', 'thrs_l_cls', 'thrs_s_opn', 'thrs_s_cls', 
                                                'start', 'end', 'dur', 'init_capital', 'final_capital', 'roi', 'roi_ann', 'volatility_ann', 'sharpe_ratio_ann', 
                                                'port_mdd', 'trade_mdd', 'trade_P10_mdd', 'win_rate', 'no_trades', 'avg_hld_dur', 'max_hld_dur', 'reward_to_risk', 'expectency'])


  df_thrs_s = df_thrs_s.sort_values(by=['roi'], ascending=False).reset_index(drop=True)

  # Backtest report
  thrs_report = {'backtest_opt_l': {'thrs_l':[df_thrs_l['thrs_l_opn'][0], df_thrs_l['thrs_l_cls'][0]], 'thrs_s':[np.nan, np.nan]},
                 'backtest_opt_s': {'thrs_l':[np.nan, np.nan], 'thrs_s':[df_thrs_s['thrs_s_opn'][0], df_thrs_s['thrs_s_cls'][0]]},
                 'backtest_0_l': {'thrs_l':[np.nan, np.nan], 'thrs_s':[np.nan, np.nan]},
                 'backtest_0_s': {'thrs_l':[np.nan, np.nan], 'thrs_s':[np.nan, np.nan]}
                }

  backtest_result = {}
  df_backtest_result = []

  for k, v in thrs_report.items():
    if k in ['backtest_opt_l', 'backtest_0_l']:
      position = ['long']
    elif k in ['backtest_opt_s', 'backtest_0_s']:
      position = ['short']
    
    df_evaluation_thrs = SMA_strategy(df_evaluation, position = position)
    backtest_evaluation = Backtest(df_evaluation_thrs, df_evaluation_thrs.index[0], df_evaluation_thrs.index[-1], col_name = None, initial_capital = initial_capital, trd_fee_frac = trd_fee_frac, hld_fee_frac_hr = hld_fee_frac_hr)
    backtest_result[k] = {'df': df_evaluation_thrs.copy(deep=True),
                          'backtest': backtest_evaluation.copy()}
    df_backtest_result.append([k] + report_summary(backtest_thrs, print_report = False, return_list = True))

  df_backtest_result = pd.DataFrame(data = df_backtest_result, columns=['name', 
                                                'start', 'end', 'dur', 'init_capital', 'final_capital', 'roi', 'roi_ann', 'volatility_ann', 'sharpe_ratio_ann', 
                                                'port_mdd', 'trade_mdd', 'trade_P10_mdd', 'win_rate', 'no_trades', 'avg_hld_dur', 'max_hld_dur', 'reward_to_risk', 'expectency'])
  return df_thrs_l, df_thrs_s, backtest_result, df_backtest_result

In [ ]:
# create strategy
def SMA_strategy(df, position = ['long'], signal = 'SMA_15', signal2 = 'SMA_50'): # position = ['long'], ['short'], ['long', 'short']

  df['Long_Position'] = np.nan
  df['Long_Price'] = np.nan
  df['Short_Position'] = np.nan
  df['Short_Price'] = np.nan

  for i in range(len(df)):
    idx = df.index[i]
    # long position =======
    if 'long' in position:
      if i == 0: 
        df.loc[idx, 'Long_Position'] = 0
        pass

      elif df[signal][i-1] < df[signal2][i-1] and df[signal][i] >= df[signal2][i]: # SMA50 cross SMA200 upward
        # trigger open long position
        df.loc[idx, 'Long_Position'] = 1
        df.loc[idx, 'Long_Price'] = df['Close'][i]

      elif (df[signal][i-1] > df[signal2][i-1] and df[signal][i] <= df[signal2][i]) or i == len(df) - 1: # SMA50 cross SMA200 downward
        # trigger close long position 
        df.loc[idx, 'Long_Position'] = 0
        df.loc[idx, 'Long_Price'] = df['Close'][i]
      
      else:
        # do nothing (continue last position)
        df.loc[idx, 'Long_Position'] = df['Long_Position'][i-1]


    # short position =======
    if 'short' in position:
      if i == 0: 
        df.loc[idx, 'Short_Position'] = 0

      elif df[signal][i-1] > df[signal2][i-1] and df[signal][i] <= df[signal2][i]: # SMA50 cross SMA200 downward
        # trigger open short position
        df.loc[idx, 'Short_Position'] = 1
        df.loc[idx, 'Short_Price'] = df['Close'][i]

      elif (df[signal][i-1] < df[signal2][i-1] and df[signal][i] >= df[signal2][i]) or i == len(df) - 1: # SMA50 cross SMA200 upward
        # trigger close short position 
        df.loc[idx, 'Short_Position'] = 0
        df.loc[idx, 'Short_Price'] = df['Close'][i]
      
      else:
        # do nothing (continue last position)
        df.loc[idx, 'Short_Position'] = df['Short_Position'][i-1]

  return df

def STO_model_results(df, date_index, initial_capital = 100):
  df_evaluation = df[OHLC_features + targets2].copy(deep=True)
  df_evaluation['%K'] = ta.momentum.StochRSIIndicator(close=df['Close'], window=n_ind, smooth1=n_ind, smooth2=n_ind, fillna=False).stochrsi_k()*100
  df_evaluation['%D'] = ta.momentum.StochRSIIndicator(close=df['Close'], window=n_ind, smooth1=n_ind, smooth2=n_ind, fillna=False).stochrsi_d()*100
  df_evaluation = df_evaluation[test_date[0]:test_date[-1]]

  # thrs_range = np.linspace(10, 90, 9)
  # thrs_range_l_opn = thrs_range
  # thrs_range_l_cls = thrs_range
  # thrs_range_s_opn = thrs_range
  # thrs_range_s_cls = thrs_range
  thrs_range_l_opn = [20]
  thrs_range_l_cls = [80]
  thrs_range_s_opn = [80]
  thrs_range_s_cls = [20]

  # Long
  df_thrs_l = []
  for thrs_l_opn in thrs_range_l_opn:
    for thrs_l_cls in thrs_range_l_cls:
      df_evaluation_thrs = STO_strategy(df_evaluation, thrs_l = [thrs_l_opn, thrs_l_cls], thrs_s = [np.nan, np.nan])
      backtest_thrs = Backtest(df_evaluation_thrs, df_evaluation_thrs.index[0], df_evaluation_thrs.index[-1], col_name = None, initial_capital = initial_capital, trd_fee_frac = trd_fee_frac, hld_fee_frac_hr = hld_fee_frac_hr)
      df_thrs_l.append([thrs_l_opn, thrs_l_cls, np.nan, np.nan] + report_summary(backtest_thrs, print_report = False, return_list = True))

  df_thrs_l = pd.DataFrame(data = df_thrs_l, columns=['thrs_l_opn', 'thrs_l_cls', 'thrs_s_opn', 'thrs_s_cls', 
                                                'start', 'end', 'dur', 'init_capital', 'final_capital', 'roi', 'roi_ann', 'volatility_ann', 'sharpe_ratio_ann', 
                                                'port_mdd', 'trade_mdd', 'trade_P10_mdd', 'win_rate', 'no_trades', 'avg_hld_dur', 'max_hld_dur', 'reward_to_risk', 'expectency'])

  df_thrs_l = df_thrs_l.sort_values(by=['roi'], ascending=False).reset_index(drop=True)

  # Short
  df_thrs_s = []
  for thrs_s_opn in thrs_range_s_opn:
    for thrs_s_cls in thrs_range_s_cls:
      df_evaluation_thrs = STO_strategy(df_evaluation, thrs_l = [np.nan, np.nan], thrs_s = [thrs_s_opn, thrs_s_cls])
      backtest_thrs = Backtest(df_evaluation_thrs, df_evaluation_thrs.index[0], df_evaluation_thrs.index[-1], col_name = None, initial_capital = initial_capital, trd_fee_frac = trd_fee_frac, hld_fee_frac_hr = hld_fee_frac_hr)
      df_thrs_s.append([np.nan, np.nan, thrs_s_opn, thrs_s_cls] + report_summary(backtest_thrs, print_report = False, return_list = True))

  df_thrs_s = pd.DataFrame(data = df_thrs_s, columns=['thrs_l_opn', 'thrs_l_cls', 'thrs_s_opn', 'thrs_s_cls', 
                                                'start', 'end', 'dur', 'init_capital', 'final_capital', 'roi', 'roi_ann', 'volatility_ann', 'sharpe_ratio_ann', 
                                                'port_mdd', 'trade_mdd', 'trade_P10_mdd', 'win_rate', 'no_trades', 'avg_hld_dur', 'max_hld_dur', 'reward_to_risk', 'expectency'])


  df_thrs_s = df_thrs_s.sort_values(by=['roi'], ascending=False).reset_index(drop=True)

  # # Backtest report
  thrs_report = {'backtest_opt_l': {'thrs_l':[df_thrs_l['thrs_l_opn'][0], df_thrs_l['thrs_l_cls'][0]], 'thrs_s':[np.nan, np.nan]},
                 'backtest_opt_s': {'thrs_l':[np.nan, np.nan], 'thrs_s':[df_thrs_s['thrs_s_opn'][0], df_thrs_s['thrs_s_cls'][0]]},
                 'backtest_0_l': {'thrs_l':[20, 80], 'thrs_s':[np.nan, np.nan]}, # normal trigger point for STO strategy = 20/80
                 'backtest_0_s': {'thrs_l':[np.nan, np.nan], 'thrs_s':[80, 20]} # normal trigger point for STO strategy = 20/80
                }

  backtest_result = {}
  df_backtest_result = []

  for k, v in thrs_report.items():
    df_evaluation_thrs = STO_strategy(df_evaluation, thrs_l = [v['thrs_l'][0], v['thrs_l'][1]], thrs_s = [v['thrs_s'][0], v['thrs_s'][1]])
    backtest_evaluation = Backtest(df_evaluation_thrs, df_evaluation_thrs.index[0], df_evaluation_thrs.index[-1], col_name = None, initial_capital = initial_capital, trd_fee_frac = trd_fee_frac, hld_fee_frac_hr = hld_fee_frac_hr)
    backtest_result[k] = {'df': df_evaluation_thrs.copy(deep=True),
                          'backtest': backtest_evaluation.copy()}
    df_backtest_result.append([k] + report_summary(backtest_thrs, print_report = False, return_list = True))

  df_backtest_result = pd.DataFrame(data = df_backtest_result, columns=['name', 
                                                'start', 'end', 'dur', 'init_capital', 'final_capital', 'roi', 'roi_ann', 'volatility_ann', 'sharpe_ratio_ann', 
                                                'port_mdd', 'trade_mdd', 'trade_P10_mdd', 'win_rate', 'no_trades', 'avg_hld_dur', 'max_hld_dur', 'reward_to_risk', 'expectency'])
  return df_thrs_l, df_thrs_s, backtest_result, df_backtest_result

# create strategy
def STO_strategy(df,  thrs_l = [0, 0], # [Open (>=) , Close (<=)]
                             thrs_s = [0, 0], signal = '%K', signal2 = '%D'):   # [Open (<=) , Close (>=)]
  df['thrs_l_opn'] = thrs_l[0]
  df['thrs_l_cls'] = thrs_l[1]
  df['thrs_s_opn'] = thrs_s[0]
  df['thrs_s_cls'] = thrs_s[1]

  df['Long_Position'] = np.nan
  df['Long_Price'] = np.nan
  df['Short_Position'] = np.nan
  df['Short_Price'] = np.nan

  for i in range(len(df)):
    idx = df.index[i]
    # long position =======
    if i == 0: 
      df.loc[idx, 'Long_Position'] = 0
      pass

    elif df[signal2][i] <= df['thrs_l_opn'][i] and df[signal][i-1] < df[signal2][i-1] and df[signal][i] >= df[signal2][i]: # %D <= thres , %K cross %D upward
      # trigger open long position
      df.loc[idx, 'Long_Position'] = 1
      df.loc[idx, 'Long_Price'] = df['Close'][i]

    elif (df[signal2][i] >= df['thrs_l_cls'][i] and df[signal][i-1] > df[signal2][i-1] and df[signal][i] <= df[signal2][i]) or i == len(df) - 1: # %D >= thres , %K cross %D downward
      # trigger close long position 
      df.loc[idx, 'Long_Position'] = 0
      df.loc[idx, 'Long_Price'] = df['Close'][i]
    
    else:
      # do nothing (continue last position)
      df.loc[idx, 'Long_Position'] = df['Long_Position'][i-1]


    # short position =======
    if i == 0: 
      df.loc[idx, 'Short_Position'] = 0

    elif df[signal2][i] >= df['thrs_s_cls'][i] and df[signal][i-1] > df[signal2][i-1] and df[signal][i] <= df[signal2][i]: # %D >= thres , %K cross %D downward
      # trigger open short position
      df.loc[idx, 'Short_Position'] = 1
      df.loc[idx, 'Short_Price'] = df['Close'][i]

    elif (df[signal2][i] <= df['thrs_s_opn'][i] and df[signal][i-1] < df[signal2][i-1] and df[signal][i] >= df[signal2][i]) or i == len(df) - 1: # %D <= thres , %K cross %D upward
      # trigger close short position 
      df.loc[idx, 'Short_Position'] = 0
      df.loc[idx, 'Short_Price'] = df['Close'][i]
    
    else:
      # do nothing (continue last position)
      df.loc[idx, 'Short_Position'] = df['Short_Position'][i-1]

  return df

# Backtest

In [ ]:
def get_backtest_result(model, X, y, date_list, original_df, scaler, scaler_label, feature_columns, target_columns, m):
  y_pred = model.predict(X, verbose=0)

  train_data_filtered = original_df[pd.to_datetime(original_df['Time']).isin(date_list)].copy()
  train_data_filtered['predict_1'] = scaler_label.inverse_transform(y_pred).reshape(-1)

  train_data_filtered['Time'] = pd.to_datetime(train_data_filtered['Time'])
  temp_df =  train_data_filtered.copy()
  temp_df = temp_df.set_index('Time')
  # global test_date
  # test_date = date_list

  temp_df['buy'] = np.nan
  temp_df['sell'] = np.nan
  temp_df['hold'] = np.nan
  temp_df['port'] = np.nan

  score,temp, m = trade_by_model(temp_df.reset_index(), m)

  # print("#"*50)
  # print("SCORE BACK TEST:")
  # print('BH', roi_bh)
  # print('MACD', roi_macd)
  # print('RSI', roi_rsi)
  # print('SMA', roi_sma)
  # print('STO', roi_sto)
  # print("MODEL", score)
  loss, mse = model.evaluate(X, y, verbose=0)
  data = {
    # 'BH':[roi_bh],
    # 'MACD':[roi_macd],
    # 'RSI':[roi_rsi],
    # 'SMA':[roi_sma],
    # 'STO':[roi_sto],
    'ROI_MODEL': [score],
    'lOSS':[loss],
    'MSE':[mse]
  } 

  return data, temp, m

In [ ]:
# session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
# sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
# tf.compat.v1.keras.backend.set_session(sess)
# df = pd.read_csv("/content/gdrive/MyDrive/2022/data_set_day.csv")
# df['Time'] = pd.to_datetime(df['Time']).dt.strftime("%Y-%m-%d 00:00:00")
# df = df.drop(columns=['index'])
# result = pd.DataFrame()
# for i in ['AMD', 'GOOGL', 'NVDA', 'OXY', 'SQQQ', 'TQQQ', 'TSLA']:
#   df2 = df[df['Stock'] == i].copy()
#   score = main(df2)
#   score['STOCK'] = i
#   result = pd.concat([result, score], axis=0)


# Prep data

In [ ]:
def df_extract_feature(df, rolling_indicator = False):
  df['Rel_Open'] = (df['Open'] - df['Close'])/df['Close']
  df['Rel_High'] = (df['High'] - df['Close'])/df['Close']
  df['Rel_Low'] = (df['Low'] - df['Close'])/df['Close']

  # Create Indicator Column

  # SMA
  df['Rel_SMA_fast'] = (ta.trend.SMAIndicator(close=df['Close'], window=n_ind, fillna=False).sma_indicator() - df['Close']) / df['Close'] 
  df['Rel_SMA_slow'] = (ta.trend.SMAIndicator(close=df['Close'], window=k_ind, fillna=False).sma_indicator() - df['Close']) / df['Close'] 

  # CCI
  df['CCI'] = ta.trend.CCIIndicator(high=df['High'], low=df['Low'], close=df['Close'], window=n_ind, constant=0.015, fillna=False).cci()

  # ROC
  df['ROC'] = ta.momentum.ROCIndicator(close=df['Close'], window=n_ind, fillna=False).roc()

  # %R
  df['%R'] = ta.momentum.WilliamsRIndicator(high=df['High'], low=df['Low'], close=df['Close'], lbp=n_ind, fillna=False).williams_r()

  # MFI
  df['MFI'] = ta.volume.MFIIndicator(high=df['High'], low=df['Low'], close=df['Close'], volume=df['Volume'], window=n_ind, fillna=False).money_flow_index()

  # VPT
  df['VPT'] = ta.volume.VolumePriceTrendIndicator(close=df['Close'], volume=df['Volume'], fillna=False).volume_price_trend()
  df.loc[df.index[0], 'VPT'] = np.nan
  df.loc[df.index[1], 'VPT'] = np.nan = np.nan # correct library bug. the first 2 row in VPT should be np.nan

  # VWAP
  df['VWAP'] = (ta.volume.VolumeWeightedAveragePrice(high=df['High'], low=df['Low'], close=df['Close'], volume=df['Volume'], window=n_ind, fillna=False).volume_weighted_average_price()- df['Close']) / df['Close']

  # MACD
  df['MACD'] = ta.trend.macd(close=df['Close'], window_slow=k_ind, window_fast=n_ind, fillna=False)

  # ADX # create warning but the results are correct
  # warnings.filterwarnings('ignore', category=RuntimeWarning)
  df['ADX'] = ta.trend.ADXIndicator(high=df['High'], low=df['Low'], close=df['Close'], window=n_ind, fillna=False).adx()
  # warnings.filterwarnings('default', category=RuntimeWarning)

  # RSI
  df['RSI'] = ta.momentum.RSIIndicator(close=df['Close'], window=n_ind, fillna=False).rsi()

  # TSI
  df['TSI'] = ta.momentum.TSIIndicator(close=df['Close'], window_slow=k_ind, window_fast=n_ind, fillna=False).tsi()

  # %K
  df['%K'] = ta.momentum.StochRSIIndicator(close=df['Close'], window=n_ind, smooth1=n_ind, smooth2=n_ind, fillna=False).stochrsi_k()

  # %D
  df['%D'] = ta.momentum.StochRSIIndicator(close=df['Close'], window=n_ind, smooth1=n_ind, smooth2=n_ind, fillna=False).stochrsi_d()

  # Bollinger Bands
  df['Rel_BB_hband'] = (ta.volatility.BollingerBands(close=df['Close'], window=n_ind, window_dev=2, fillna=False).bollinger_hband() - df['Close']) / df['Close']
  df['Rel_BB_lband'] = (ta.volatility.BollingerBands(close=df['Close'], window=n_ind, window_dev=2, fillna=False).bollinger_lband() - df['Close']) / df['Close']

  # ATR
  df['Rel_ATR'] = ta.volatility.AverageTrueRange(high=df['High'], low=df['Low'], close=df['Close'], window=n_ind, fillna=False).average_true_range() / df['Close']

  # UI
  df['UI'] = ta.volatility.UlcerIndex(close=df['Close'], window=n_ind, fillna=False).ulcer_index()

  #ADI (Accumulation/Distribution Index)
  # df['ADI'] = ta.volume.AccDistIndexIndicator(high=df['High'], low=df['Low'], close=df['Close'], volume=df['Volume'], fillna=True).acc_dist_index()

  # # # OBV
  # df['OBV'] = ta.volume.OnBalanceVolumeIndicator(close=df['Close'], volume=df['Volume'], fillna=False).on_balance_volume()

  # CMF
  df['CMF'] = ta.volume.ChaikinMoneyFlowIndicator(high=df['High'], low=df['Low'], close=df['Close'], volume=df['Volume'], window=n_ind, fillna=False).chaikin_money_flow()

  # FI 
  df['FI'] = ta.volume.ForceIndexIndicator(close=df['Close'], volume=df['Volume'], window=n_ind, fillna=False).force_index()

  return df


In [ ]:
def Xy4nn(df, feature_columns, target_columns, T):
  X, y = [], []
  date_list = []
  for i in range(df[target_columns].shape[0] - (T-1)):
    date_list.append(df[['Time']].iloc[i + (T-1)].values)
    X.append(df[feature_columns].iloc[i:i+T].values)
    y.append(df[target_columns].iloc[i + (T-1)].values)

  X, y = np.array(X), np.array(y).reshape(-1,len(target_columns))
  return X, y, date_list

In [ ]:
import itertools
from itertools import permutations
from keras.utils.vis_utils import plot_model

In [ ]:
def df_identifying_label(df, span):
  df['Smoothed_Price*'] = smooth_data_savgol_2(np.array(df['Close']), span)
  # df['Smoothed_Price*'] = smooth_data_savgol_2(np.array(df['Smoothed_Price*']), span)
  # df['Smoothed_Price*'] = smooth_data_savgol_2(np.array(df['Smoothed_Price*']), span)
  df['Smoothed_Return*'] = df['Smoothed_Price*'].pct_change(1).shift(periods=-1)
  return df

def smooth_data_savgol_2(arr, span):  
    return savgol_filter(arr, span * 2 + 1, 1)

In [ ]:
import keras_tuner as kt
from keras.layers import Bidirectional

In [ ]:
def get_backtest_technical_result(X, y,date_list, original_df, scaler, scaler_label, feature_columns, target_columns, m):
  # y_pred = model.predict(X, verbose=0)

  train_data_filtered = original_df[pd.to_datetime(original_df['Time']).isin(date_list)].copy()
  # train_data_filtered['predict_1'] = scaler_label.inverse_transform(y_pred).reshape(-1)
  train_data_filtered['Time'] = pd.to_datetime(train_data_filtered['Time'])
  temp_df =  train_data_filtered.copy()
  temp_df = temp_df.set_index('Time')
  global test_date
  test_date = date_list

  df_thrs_l_rsi, df_thrs_s_rsi, backtest_result_rsi, df_backtest_result_rsi = RSI_model_results(temp_df, test_date, initial_capital = float(m['RSI'][0]))
  df_thrs_l_bh, df_thrs_s_bh, backtest_result_bh, df_backtest_result_bh = BH_model_results(temp_df, test_date, initial_capital = float(m['BH'][0]))
  df_thrs_l_macd, df_thrs_s_macd, backtest_result_macd, df_backtest_result_macd = MACD_model_results(temp_df, test_date, initial_capital = float(m['MACD'][0]))
  df_thrs_l_sma, df_thrs_s_sma, backtest_result_sma, df_backtest_result_sma = SMA_model_results(temp_df, test_date, initial_capital = float(m['SMA'][0]))
  df_thrs_l_sto, df_thrs_s_sto, backtest_result_sto, df_backtest_result_sto = STO_model_results(temp_df, test_date, initial_capital = float(m['STO'][0]))
  roi_list = list()
  roi_rsi = df_thrs_l_rsi['final_capital'].values[0]
  roi_list.append(roi_rsi)
  roi_bh = df_thrs_l_bh['final_capital'].values[0]
  roi_list.append(roi_bh)
  roi_macd = df_thrs_l_macd['final_capital'].values[0]
  roi_list.append(roi_macd)
  roi_sma = df_thrs_l_sma['final_capital'].values[0]
  roi_list.append(roi_sma)
  roi_sto = df_thrs_l_sto['final_capital'].values[0]
  roi_list.append(roi_sto)
    # roi_list

  data = {
    'BH':[roi_bh],
    'MACD':[roi_macd],
    'RSI':[roi_rsi],
    'SMA':[roi_sma],
    'STO':[roi_sto],
  } 
  return data, df_thrs_l_bh

def trade_by_model(temp, start_m=100, hold=0):
  temp['Close_predict'] = temp['Close']+(temp['Close']*temp['predict_1'])
  m = start_m
  count_buy = 0
  count_sell = 0
  for index, row in temp.iterrows():
    status_buy = 0
    status_sell = 0

    # if row['sum_p'] >= 3: 
    #   status_buy = 1
    # elif row['sum_n'] < 3:
    #   status_sell = 1
    #if row['Close_predict'] >= row['High']:
    if row['Close_predict'] >= row['High']:
      status_buy = 1
    elif row['predict_1'] < -0.001:
      status_sell = 1

    # if (row['smoothed_return_close_predict_n1'] > 0) & (row['smoothed_return_close_predict_n2'] > 0) & (row['smoothed_return_close_predict_n3'] > 0):
    #   status_buy = 1
    # elif (row['smoothed_return_close_predict_n1'] < 0) & (row['smoothed_return_close_predict_n2'] < 0) & (row['smoothed_return_close_predict_n3'] < 0):
    #   status_sell = 1

    if status_buy == 1:
      if hold == 1:
        pass
      else:
        count_buy = count_buy + 1
        temp.loc[index:]['buy'] = 1
        temp['buy'] = np.where(temp['Time']==row['Time'],1,temp['buy'])
        current_price = row['Close']
        hold = 1
        

    if status_sell == 1:
      if hold == 0:
        pass
      else:
        count_sell = count_sell +1
        temp.loc[index:]['sell'] = 1
        temp['sell'] = np.where(temp['Time']==row['Time'],1,temp['sell'])
        m = (row['Close']/current_price)*m
        hold = 0

    temp['port'] = np.where(temp['Time']==row['Time'],m,temp['port'])
    temp['hold'] = np.where(temp['Time']==row['Time'],hold,temp['hold'])
  # print(count_buy)
  # print(count_sell)
  if hold == 1:
    count_sell = count_sell +1
    temp.loc[index,'sell'] = 1
    temp['port'] = np.where(temp['Time']==row['Time'],(row['Close']/current_price)*m,temp['port'])
    temp['hold'] = np.where(temp['Time']==row['Time'],0,temp['hold'])
    return ((((row['Close']/current_price)*m)-start_m)/start_m)*100, temp, m
  else:
    return ((m-start_m)/start_m)*100, temp, m

# sandbox

In [ ]:
def model_builder(hp):

  input1 = Input(shape=(32,28))
  lstm_hp_units = hp.Choice('lstm_units', values=[32, 64 ,128 ,256])
  cnn_hp_units = hp.Choice('cnn_units', values=[32, 64 ,128 ,256])
  kernel_size_list = hp.Int('kernal_units', min_value=3, max_value=9, step=1)
  drop_out_hp_units = hp.Choice('drop_out_units', values=[0.3, 0.35 ,0.4 ,0.45])
  x = Conv1D(filters=cnn_hp_units, kernel_size=kernel_size_list, activation='relu')(input1)
  x = Dropout(drop_out_hp_units)(x)
  # x = Bidirectional(LSTM(lstm_hp_units, return_sequences=True, activation='relu'))(x)
  # x = Dropout(drop_out_hp_units)(x)
  # x = Bidirectional(LSTM(lstm_hp_units, return_sequences=True))(x)
  # x = Dropout(drop_out_hp_units)(x)
  # x = LSTM(lstm_hp_units, return_sequences=True, activation='relu')(input1)
  # x = Dropout(drop_out_hp_units)(x)
  x = LSTM(lstm_hp_units, return_sequences=True, activation='relu')(x)
  x = Dropout(drop_out_hp_units)(x)
  # x = Conv1D(filters=cnn_hp_units, kernel_size=kernel_size_list, activation='relu')(x)
  # x = Dropout(drop_out_hp_units)(x)
  # x = LSTM(lstm_no_node, return_sequences=True)(x)
  # x = Dropout(0.4)(x)
  x = Flatten()(x)
  x = BatchNormalization()(x)
  dense_list = hp.Int('dense_units', min_value=32, max_value=128, step=32)
  x = Dense(dense_list)(x)
  # x = Dropout(drop_out_hp_units)(x)
  out1 = Dense(1, activation='sigmoid', name='class1')(x)

  # model = keras.Sequential()
  # model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # # Tune the number of units in the first Dense layer
  # # Choose an optimal value between 32-512
  # hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  # model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  # model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])#, 1e-5, 1e-6, 1e-7, 1e-8])#, 1e-5, 1e-6, 1e-7, 1e-8])
  model = Model(inputs=input1, outputs=[out1])

  model.compile(optimizer='SGD',
                #Adam(learning_rate=hp_learning_rate),
            loss='MeanSquaredError',
            metrics='MeanSquaredError')#[tf.keras.metrics.RootMeanSquaredError()])
  # model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
  #               loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  #               metrics=['accuracy'])

  return model

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.preprocessing import StandardScaler
feature_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 
      'Rel_Open', 'Rel_High', 'Rel_Low', 'Rel_SMA_fast', 'Rel_SMA_slow',
      'CCI', 'ROC', '%R', 'MFI', 'VPT', 'VWAP', 'MACD', 'ADX', 'RSI', 'TSI',
      '%K', '%D', 'Rel_BB_hband', 'Rel_BB_lband', 'Rel_ATR', 'UI', 'CMF',
      'FI']

target_columns = ['label_1']

In [ ]:
data = pd.read_csv("/content/gdrive/MyDrive/2023/data_set_1hr_2023.csv")
result_summary = pd.DataFrame()
#'TQQQ', 'SQQQ', 'MRNA', 'APA', 'NFLX', 'MOS', 'AMD', 'NVDA','ABNB', 
for stock_name in ['TQQQ', 'SQQQ', 'MRNA', 'APA', 'NFLX', 'MOS', 'AMD', 'NVDA', 'OXY', 'DVN', 'TSLA', 'GOOGL']:
       #'OXY', 'DVN', '1TSLA', 'GOOGL']:
  for test_year in [2020, 2021, 2022]:
    print(stock_name, test_year)
    tf.compat.v1.keras.backend.set_session(sess)
    seed_value = 42 # seed for all random function
    os.environ['PYTHONHASHSEED']=str(seed_value)
    random.seed(seed_value)
    np.random.seed(seed_value)
    tf.random.set_seed(seed_value)
    tf.keras.utils.set_random_seed(0) 
    df = data[data['Stock'] == stock_name].copy()     
    df['year'] = pd.to_datetime(df['Time']).dt.year
    df['label_1'] = df['Close'].pct_change(1).shift(periods=-1)
    # df = df_identifying_label(df, 1)
    df = df_extract_feature(df)
    df = df.dropna()
    df.reset_index(drop=True, inplace=True)
    df['Time'] = pd.to_datetime(df['Time'])
    df['number_of_day'] = df['Time'].dt.day 
    
    scaler = StandardScaler()
    scaler_label = MinMaxScaler()
    original_df = df.copy()
    df['month'] = pd.to_datetime(df['Time']).dt.month

    TEST_YEAR = test_year
    TRAIN_YEAR = TEST_YEAR - 1
    VALID_YEAR = TRAIN_YEAR
    # print("Train:", TRAIN_YEAR)#, '-', VALID_YEAR - 1)
    # print("Valid:", VALID_YEAR)
    # print("Test:", TEST_YEAR)

    train_data = df[(df['year'] == TRAIN_YEAR)].copy()# & (df['year'] < VALID_YEAR)].copy()
    val_data = df[df['year'] == VALID_YEAR].copy()
    test_data = df[df['year'] == TEST_YEAR].copy()

    # print(train_data.shape[0])
    # print(val_data.shape[0])
    # print(test_data.shape[0])

    train_data[feature_columns] = scaler.fit_transform(train_data[feature_columns])
    train_data[target_columns] = scaler_label.fit_transform(train_data[target_columns])

    val_data[feature_columns] = scaler.transform(val_data[feature_columns])
    val_data[target_columns] = scaler_label.transform(val_data[target_columns])

    # train_data = train_data.query("month == 11").query("month == 12 & number_of_day < 15").copy()
    # val_data = val_data.query("month == 12").copy()

    # #temp1 = train_data.query("month == 11")
    # train_data = train_data.query("month == 12 & number_of_day <= 22").copy()
    # #train_data = pd.concat([temp1, temp2])
    # val_data = val_data.query("month == 12 & number_of_day > 22").copy()

    train_data.reset_index(inplace=True)
    start_index_train = train_data.query("month == 12 & number_of_day <= 22").index[0]-T
    end_index_train = train_data.query("month == 12 & number_of_day <= 22").index[-1]
    train_data = train_data[start_index_train:end_index_train]

    val_data.reset_index(inplace=True)
    start_index_val = val_data.query("month == 12 & number_of_day > 22").index[0]-T
    end_index_val = val_data.query("month == 12 & number_of_day > 22").index[-1]
    val_data = val_data[start_index_val:end_index_val]

    # print(stock_name)
    # print(test_year)
    # print(train_data.shape[0])
    # print(val_data.shape[0])
    # print(test_data.shape[0])

    # temp = {
    #     'stock':[stock_name],
    #     'test_year':[test_year],
    #     'train':[train_data.shape[0]],
    #     'valid':[val_data.shape[0]],
    #     'test':[test_data.shape[0]]
    # }
    # result_summary = pd.concat([result_summary, pd.DataFrame(temp)])

    X_train, y_train, train_date = Xy4nn(train_data, feature_columns, target_columns, T)
    X_val, y_val, val_date = Xy4nn(val_data, feature_columns, target_columns, T)
    X_test, y_test, test_date = Xy4nn(test_data, feature_columns, target_columns, T)
    n_epochs = 30
    tuner = kt.Hyperband(model_builder,
                        objective='val_loss',
                        max_epochs=10,
                        factor=3,
                        directory='my_dir',
                        project_name='intro_to_kt', overwrite=True)

    early_stop = EarlyStopping(monitor='val_loss', min_delta=0,
                              patience=30, verbose=0, mode='auto',
                              baseline=None, restore_best_weights=True)

    tuner.search(X_train, y_train, epochs=n_epochs, validation_split=0.0,  validation_data=(X_val, y_val), callbacks=[early_stop], batch_size=32)

    # Get the optimal hyperparameters
    best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

    # print(best_hps.get('dense_units'))
    # print(best_hps.get('lstm_units'))
    # print(best_hps.get('cnn_units'))

    # Build the model with the optimal hyperparameters and train it on the data for 50 epochs
    model = tuner.hypermodel.build(best_hps)
    history = model.fit(X_train, y_train, epochs=n_epochs, validation_split=0.0,  validation_data=(X_val, y_val), batch_size=32)

    val_acc_per_epoch = history.history['val_loss']
    best_epoch = val_acc_per_epoch.index(min(val_acc_per_epoch)) + 1
    # print('Best epoch: %d' % (best_epoch,))

    result = dict()
    train_date = np.array(train_date).reshape(-1)
    val_date = np.array(val_date).reshape(-1)
    test_data_date = np.array(test_date).reshape(-1)
    result_train, temp_train, _ = get_backtest_result(model, X_train, y_train, train_date, original_df, scaler, scaler_label, feature_columns, target_columns, 100)
    result_val, temp_val, _ = get_backtest_result(model, X_val, y_val, val_date, original_df, scaler, scaler_label, feature_columns, target_columns, 100)
    result_test, temp_test, _ = get_backtest_result(model, X_test, y_test, test_data_date, original_df, scaler, scaler_label, feature_columns, target_columns, 100)

    result_data = {
        'train':result_train['ROI_MODEL'],
        'valid':result_val['ROI_MODEL'],
        'test':result_test['ROI_MODEL'],
        'train_mse':result_train['MSE'],
        'valid_mse':result_val['MSE'],
        'test_mse':result_test['MSE'],
    }
    result = pd.DataFrame(result_data)

    result['cnn_node'] = best_hps.get('cnn_units')
    result['lstm_no_node'] = best_hps.get('lstm_units')
    result['kernal_size'] = best_hps.get('kernal_units')
    result['drop_out'] = best_hps.get('drop_out_units')
    result['stock'] = stock_name
    result['test_year'] = test_year
    result_summary = pd.concat([result_summary, result])
    #result_summary.to_csv("/content/gdrive/MyDrive/2023/1hr/lstm-cnn/result_model20230321_cnn_lstm.csv", index=False)
    # tf.compat.v1.keras.backend.clear_session()
    tf.keras.backend.clear_session()


TQQQ 2020
TQQQ 2021
TQQQ 2022
SQQQ 2020
SQQQ 2021
SQQQ 2022
MRNA 2020
MRNA 2021
MRNA 2022
APA 2020
APA 2021
APA 2022
NFLX 2020
NFLX 2021
NFLX 2022
MOS 2020
MOS 2021
MOS 2022
AMD 2020
AMD 2021
AMD 2022
NVDA 2020
NVDA 2021
NVDA 2022
OXY 2020
OXY 2021
OXY 2022
DVN 2020
DVN 2021
DVN 2022
TSLA 2020
TSLA 2021
TSLA 2022
GOOGL 2020
GOOGL 2021
GOOGL 2022
